# 高度なプロンプト：Chain of Thought and ReAct (Reasoning + Acting)

In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

このノートブックは、**Chain of Thought and ReAct (Reasoning + Acting)に基づいています** [Applied-Ai-Engieering-examples](https://github.com/googlecloudplatform/applied-ai-engineering-サンプル)GitHubリポジトリ。このリポジトリには、Google Cloud Applied AI Engineeringチームが開発したリファレンスガイド、青写真、コードサンプル、および実践的なラボが含まれています。

# パート0：はじめに

このノートブックのターゲットオーディエンスは、タスク、ワークフロー、プロセス、機能などを繰り返し実行するためのエンジニアリングプロンプトです。安定性とパフォーマンスは、1回限りのニーズを求めるよりも重要です。

このノートブックは、2つの強力なLLMプロンプト戦略をカバーしています。

React（およびそのバリアント）は、幻覚を最小限に抑えながらLLMの推論を改善するための現在の最先端のプロンプト技術です。

このノートブックの4つの部分は次のとおりです。

1. 思考のチェーンプロンプト：LLM出力を改善するために推論の言語説明を使用します。

1. アクション、検索、ツールの使用：LLMSが外部システムとどのように相互作用するか。

1. 反応（推論 +演技）プロンプト：外部システムの相互作用と考えられたチェーンプロンプトの書面による推論の説明を組み合わせます。

1. Langchain and React：Langchain React Agentを使用するときに何を期待するか。

このノートブックはColabでテストされました。

## このノートブックの使用方法

* 最初にパート0を実行します。
* パート1〜4それぞれパート0のコードに依存しますが、他の以前のパートのコードに依存しません。

##前提条件

-  LLMS（大規模な言語モデル）の理解：
-  LLMとは何か、そしてそれらがどのように機能するか。
-LLMSは、次のトークンの繰り返し予測因子として。
-LLM予測は、トレーニングデータとの類似性を最大化します。
-  LLMプロンプトの経験：
- 言語モデルを「プロンプト」することの意味。[推奨リソース](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/introduction-plompt-design)。
-  [ゼロショット、ワンショット、少数のショット](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/introduction-plompt-design#include-xamplesの違い)プロンプト、およびパフォーマンスと堅牢性を最大化するために、少数のショットプロンプトが不可欠である理由を理解すること。
-Google Cloud Vertex LLMSに基本的な知識。[推奨リソース](https://cloud.google.com/vertex-ai/docs/generative-ai/start/quickstarts/api-quickstart)
-  Langchainとは何か、それが解決することを目指している問題を知ってください。
-  [推奨リソース](https://python.langchain.com/docs/get_started/introduction)および[Tutorials](https://github.com/googlecloudplatform/generative-ai/tree/main/language/orchestration/langchain)。

##キー用語

一貫性のために、このノートブックは特定の方法で次の用語を使用します。

**プロンプト**：テンプレートに挿入される値に関係なく、コールのパフォーマンスと堅牢性を最大化する特定の手法を使用して作成されたテンプレートLLMコール。

**LLMコール**：LLMにテキストを送信します。

**LLM応答**：LLMによって予測されたテキスト、LLMコールを行うときにLLMから戻ってくるもの。

**チェーン/チェーン**コンテキストに応じて：
* 紹介されたチェーンのプロンプト、論理的に連続的な推論ステップ。
* LLMシステムでは、LLMへの連続呼び出し。各コールは前のコールの応答に依存します。

**exemplar**：1つまたは少数のプロンプトの「例」。
* 従来のMLの意味での「例」との混乱、つまり「データの一部」（「トレーニングの例」など）を避けるために使用されます。

##参照

* 小島、タケシ、他「大規模な言語モデルはゼロショットの推論者です。」神経情報処理システムの進歩35（2022）：22199-22213。[link](https://arxiv.org/abs/2205.11916)（アクセス2023 09 22）
* Wang、Xuezhi、et al。「自己整合性は、言語モデルの一連の思考推論を改善します。」arxiv preprint arxiv：2203.11171（2022）。[link](https://arxiv.org/abs/2203.11171)（アクセス2023 09 03）。
*ウェイ、ジェイソン他「考えられたチェーンプロンプトは、大規模な言語モデルで推論を引き出します。」神経情報処理システムの進歩35（2022）：24824-24837。[link](https://arxiv.org/abs/2201.11903)（アクセス2023 09 03）。
* Yao、Shunyu、et al。「反応：言語モデルでの推論と行動の相乗効果。」Arxiv Preprint arxiv：2210.03629（2022）。[link](https://arxiv.org/abs/2210.03629)（アクセス2023 09 03）。

## セットアップ - このコードを最初に実行してください！

In [2]:
# Tested with these package versions.
# Note this notebook uses matplotlib.pyplot. This is in the default Colab
#   runtime, but you may need to install it in other notebook environments.
!pip install --user langchain==0.0.316 google-cloud-aiplatform==1.35.0 prettyprinter==0.18.0 wikipedia==1.4.0 numexpr

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.6 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=1bf13cee4e160c8bab32308753a269243d24160e52f8ddfe2c2ec24bf96a9ee5
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warnin

**さらに進む前にランタイムを再起動してください**

ランタイムが削除されていない限り（再起動しても）、この以前のセルを再実行する必要はありません。

ランタイムが再起動した場合、パート0の残りのセルを再実行します。

Colabを使用している場合は、次のセルでコードを実行します。[Vertex aiisifigine]を使用するには、Googleクラウド[プロジェクト](https://cloud.google.com/resource-manager/docs/creating-managing-projects#identifying_projects)にアクセスできるアカウントでポップアップをフォローし、認証します。llms](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview)。

Colab以外の場所でこのノートブックを実行している場合は、環境に適切なGoogleクラウドアクセスがあることを確認してください。それがあなたにとって新しい概念である場合は、[あなたのローカル環境のアプリケーションデフォルトの資格情報](https://cloud.google.com/docs/authentication/provide-credentials-adc#local-dev)を調べることを検討してください。より多くの認証オプションについて説明します[こちら](https://cloud.google.com/docs/authentication)。

Google Cloudをまったく初めて使用する場合は、[開始](https://cloud.google.com/docs/get-started)。

In [ ]:
# Colab authentication.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()
    print('Authenticated')

Google CloudプロジェクトIDを次のセルに設定します。

In [ ]:
PROJECT_ID = "YOUR_PROJECT_ID_HERE"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
# Code examples may misbehave if the model is changed.
MODEL_NAME = "text-bison@001"

In [ ]:
# Set up Vertex PaLM API.
import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project=PROJECT_ID,
              location=LOCATION)
parameters = {
    "temperature": 0,
    "max_output_tokens": 1024,
    "top_p": 0.8,
    "top_k": 40
}

model = TextGenerationModel.from_pretrained(MODEL_NAME)

この関数は、ノートブック全体で使用され、完全なLLMコールと応答を表示します。

In [ ]:
def call_llm(model, parameters, llm_call, show_activity = True):
  response = model.predict(llm_call, **parameters).text

  if show_activity:
    BOLD = "\033[1m"
    UNFORMAT = "\033[0m\x1B[0m"
    print(f"{BOLD}The call to the LLM:{UNFORMAT}\n{llm_call}\n")
    print(f"{BOLD}The response:{UNFORMAT}")
    print(response)

  return response  # Return to `_` if not needed.

In [ ]:
# Wrap code cell output to improve notebook readability.
# Source: https://stackoverflow.com/questions/58890109/line-wrapping-in-collaboratory-google-results/61401455#61401455
from IPython.display import HTML, display

def set_css(arg):
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# パート1：考え方のチェーンプロンプト

LLMSにとって、チェーンはファッショナブルなアクセサリー以上のものです。

<img src = "https://raw.githubusercontent.com/GoogleCloudPlatform/specialized-training-content/184be57c9ff4de18e20f3fdfbe1ef7fb35ce023f/courses/generative_ai/app_dev_llm/images/1-chains.png">

## 概要

考え方のプロンプトでは、目的の出力に到達するための推論ステップを示す1つまたは少数のショットの模範を提供します。これは、標準の1または少数のショットプロンプトとは異なり、模範が入力と正しい出力のみを示します。

思考の連鎖模範で提供する推論の内訳は、人が問題や仕事を通して考えている自然言語の内部モノローグに似ています。

「内部モノローグ」が奇妙な概念である場合、問題を解決したり、タスクを達成したりするために自分の考えを言語化する方法を考えてください。たとえば、あなたは夕食を作っています：

「OK OK私はセロリを切り刻みました。今、私は鶏肉を始める必要があります。オーブンはオンですか？オーブンの予熱を始めましょう。待って、どの温度？もう一度レシピをチェックする必要があります... `` `

この「内部モノローグ」または「内部スピーチ」は、タスクの次に何が起こるべきかを特定することにより、これまで見たことのない新しい問題に問題解決パターンを適用することを容易にします。

テキスト推論の「内部独白」を含む模範を使用してLLMを呼び出すことにより、LLMは同様のテキスト推論を含む応答を生成します。LLMに応答の一部として推論テキストが生成されると、応答が目的の出力で終了する可能性が高くなります。

応答の推論ステップ
また、LLMが最終出力にどのように到着したかの解釈可能性を提供します。

##思考の基本の連鎖

数学の単語の問題は、数学的および論理的に単純なものであるが、推論の複数のステップが必要なため、良いチェーンのデモンストレーションです。

この例（思考の連鎖から[紙](https://arxiv.org/pdf/2201.11903.pdf)から）誤った答えに注意してください。

In [ ]:
question = """Q: ロジャーはテニスボールを5個持っている。彼はさらに2つのテニスボール缶を買った。
それぞれの缶には3個のテニスボールが入っている。ロジャーは今何個のテニスボールを持っていますか？
A: 答えは11個。
Q：食堂には23個のリンゴがありました。
ランチに20個使い、さらに6個買ったとすると、何個のリンゴがあるでしょうか？
A:"""

_ = call_llm(model, parameters, question)

一連の思考を含めるために模範を書き直すことは、LLMに、質問を複数の単純な推論のステップに分解する方法を示しています。

モデル応答は、同様の思考の連鎖に従い、正解の可能性を高めます。

In [ ]:
question = """Q: ロジャーはテニスボールを5個持っている。彼はさらに2つのテニスボール缶を買った。
それぞれの缶には3個のテニスボールが入っている。ロジャーは今何個のテニスボールを持っていますか？
A: ロジャーは5個のボールから始めました。テニスボール3個入りの缶を2つ
は6個です。5 + 6 = 11. 答えは11。
Q：食堂には23個のリンゴがありました。
ランチに20個使い、さらに6個買ったとすると、何個のリンゴがあるでしょうか？
A:"""

_ = call_llm(model, parameters, question)

思考の連鎖には、各推論ステップからのフォローするステップと中間出力/結論を説明するテキストの両方が含まれています。

以下のコードの「質問」変数を変更して、さまざまな質問を試してみてください。

In [ ]:
question = """ノムフンドは準備書面を書く。
準備書面には3つのセクションがあり、それぞれ4時間かかる。
今週は3つの準備書面を書きました。何時間かかりましたか？"""

one_shot_exemplar = """Q: ロジャーはテニスボールを5個持っている。
彼はさらに2つのテニスボール缶を買った。
それぞれの缶には3個のテニスボールが入っている。現在、彼は何個のテニスボールを持っていますか？
A: ロジャーは5個のボールから始めました。テニスボール3個入りの缶を2つ
は6個です。5 + 6 = 11. 答えは11です。
Q: """

# Prepending the one shot exemplar before the question we want answered.
llm_call = f"{one_shot_exemplar}{question}\nA:"
_ = call_llm(model, parameters, llm_call)

LLM応答は通常、模範の推論スタイルを模倣します。これは、模範の推論のチェーンがタスクに適している場合、最高のパフォーマンスを得ることができます。

以下のセルを比較してください。

In [ ]:
# Correct answer: 360, 375.
question = """高効率工場は1日当たり100台を生産する。
中効率工場は1日当たり60台を生産する。
低効率の工場は1日当たり30個を生産する。
メガコープは5つの工場を所有している。3つは高効率、2つは低効率である。
明日、低効率工場を中効率工場に改修する。
そして、残りの低効率工場では、生産量が半分になるような停電が発生する。
今日は何台生産できるだろうか？明日は何台？"""

one_shot_exemplar = """Q: ロジャーはテニスボールを5個持っている。
彼はさらに2つのテニスボール缶を買った。
それぞれの缶には3個のテニスボールが入っている。現在、彼は何個のテニスボールを持っていますか？
A: ロジャーは5個のボールから始めました。テニスボール3個入りの缶を2つ
は6個です。5 + 6 = 11. 答えは11です。
Q: """

llm_call = f"{one_shot_exemplar}{question}\nA:"
_ = call_llm(model, parameters, llm_call)

出力の間違いに注意してください。LLM応答は、明日まだ実行されている3つの高効率工場を考慮していません。

このタスクでは、さまざまな測定単位（テニスボール缶対工場出力）への接続と、数日間の数の持ち運びを含む推論ステップを使用して、一連の思考を使用することをお勧めします。

In [ ]:
better_one_shot_exemplar = """Q: 大きなテニスボール缶には5個のボールが入っている。
小さなテニスボール缶には3個のボールがある。
ロジャーは今日、大きい缶を3つ、小さい缶を2つ持っている。
明日、彼は賭けに勝ち、小さい缶を1つ大きい缶に変えた。
ロジャーが今日持っているボールは何個か？明日は何個？
A: 大缶3個は3 * 5 = 15個のテニスボール。
小さい缶2個は、2 * 3 = 6個のテニスボール。
今日、ロジャーは15 + 6 = 21個のテニスボールを持っている。
明日のトレードは、小さなテニスボール缶を1つ失い、大きな缶を1つ得ることを意味する。
ロジャーは昨日持っていた缶をまだ持っている。
昨日の小さい缶2個 - 1 = 小さい缶1個
昨日の大缶3個＋1個＝大缶4個
大きい缶4個は、4 * 5 = 20個のテニスボール。
小さい缶1個は、1 * 3個のテニスボール。
明日、ロジャーは20 + 3 = 23個のテニスボールを持っている。
Q: """

llm_call = f"{better_one_shot_exemplar}{question}\nA:"
_ = call_llm(model, parameters, llm_call)

##思考ユースケースの連鎖

数学の単語の問題はあまり役に立たないかもしれませんが、一連の思考は他のタイプの問題でうまく機能します。

思考の連鎖からのいくつかの例[紙](https://arxiv.org/pdf/2201.11903.pdf)は情報を操作し、妥当性を評価し、指示を与え、テキストを変更/理解し、状態を追跡しています。

<img src = "https://raw.githubusercontent.com/GoogleCloudPlatform/specialized-training-content/184be57c9ff4de18e20f3fdfbe1ef7fb35ce023f/courses/generative_ai/app_dev_llm/images/2-cot.png">

思考の連鎖によく反応する他のタイプのタスクは次のとおりです。
* データの変換と濃縮。
* データの解釈。
* コード生成。
* テキストの品質の評価（LLM応答の品質の評価を含む）。
* 合成データの作成。

一般的に、いくつかの簡単なステップを「話す」ことによって解決されるあらゆる種類の問題は、思考候補の良いチェーンです。

より複雑な思考の使用の使用のために、模範全体であなたの考え方の推論スタイルがより一貫しているほど、LLMはその応答において同じスタイルの推論に従う可能性が高くなります。これは次の2つの例に注意してください。

####例：テーブルの理解

In [ ]:
# The correct answer is Post-War British Literature.
question = """
| Book Name | Edition | ISBN | Publisher | Aug 1 Amazon Avg New Price | Aug 1 Amazon Avg Used Price | Aug 1 Abebooks Avg New Price | Aug 1 Abebooks Avg Used Price | Sep 1 Amazon Avg New Price | Sep 1 Amazon Avg Used Price | Sep 1 Abebooks Avg New Price | Sep 1 Abebooks Avg Used Price |
|---|---|---|---|---|---|---|---|---|---|---|---|
| Physics for Computer Scientists | 10th | 978-1-118-56906-1 | Pearson Education | $149.99 | $79.99 | $142.94 | $66.94 | $129.99 | $59.99 | $139.94 | $56.94 |
| Fundamentals of Calculus | 8th | 978-0-470-45831-0 | John Wiley & Sons | $139.99 | $99.99 | $137.94 | $87.94 | $129.99 | $79.99 | $129.94 | $76.94 |
| Post-War British Literature | 2nd | 978-0-300-08897-2 | Oxford University Press | $129.99 | $89.99 | $122.94 | $74.94 | $119.99 | $74.99 | $124.94 | $71.94 |
| Modern Religions: An Overview | 3rd | 978-0-19-992545-3 | Oxford University Press | $119.99 | $79.99 | $117.94 | $72.94 | $114.99 | $69.99 | $114.94 | $66.94 |
| The Norton Introduction to Literature | 11th | 978-0-393-45078-1 | W. W. Norton & Company | $129.99 | $89.99 | $122.94 | $74.94 | $119.99 | $74.99 | $124.94 | $71.94 |
| The Norton Anthology of American Literature | 9th | 978-0-393-93750-8 | W. W. Norton & Company | $179.99 | $139.99 | $174.94 | $127.94 | $169.99 | $124.99 | $174.94 | $121.94 |
| The Norton Anthology of World Literature | 8th | 978-0-393-92855-6 | W. W. Norton & Company | $179.99 | $139.99 | $174.94 | $127.94 | $169.99 | $124.99 | $174.94 | $121.94 |
| The Elements of Style | 5th | 978-0-205-11265-3 | Longman | $119.99 | $79.99 | $117.94 | $72.94 | $114.99 | $69.99 | $114.94 | $66.94 |

8月から9月にかけてAmazonで最も古本価格が下落したOxfordの本は？
"""

context = """表に関する質問に答える。
すべての質問は、表の中の事実によって裏付けられていなければならない。
すべての推論はステップ・バイ・ステップで行うこと。
理由を説明する。
複数の行を見るときは、それぞれの行の理由を1つずつ説明すること。
"""

llm_call = f"{context}\n{question}\nAnswer:"
_ = call_llm(model, parameters, llm_call)

次に、いくつかの模範を追加します。

模範は質問とは異なるソーステーブルを使用しているが、考え方のチェーンの推論はまだ機能していることに注意してください。

In [ ]:
few_shot_exemplar = """
Table:
| Item Name | SKU | Vendor | Aug 1 Inventory | Sep 1 Inventory | Sale Count |
|---|---|---|---|---|---|
| iPhone 13 Pro Max | MGL83LL/A | Apple | 100 | 80 | 17 |
| iPhone 13 Pro | MLL03LL/A | Apple | 50 | 40 | 9 |
| iPhone 13 | MLKG3LL/A | Apple | 25 | 20 | 4 |
| Samsung Galaxy S22 Ultra | SM-S908U | Samsung | 100 | 80 | 19 |
| Samsung Galaxy S22 Plus | SM-S906U | Samsung | 50 | 40 | 10 |
| Samsung Galaxy S22 | SM-S901U | Samsung | 25 | 20 | 5 |
| Google Pixel 6 Pro | GA01314-US | Google | 100 | 80 | 20 |

Question:
8月に最も売れたiPhoneは？
答え 1つ1つの商品を見て、それがiPhoneかどうかを判断する必要があります。
iPhoneのアイテムのみを考慮する。
iPhoneのアイテムとは、iPhone 13 Pro Max、iPhone 13 Pro、iPhone 13です。
それぞれのiPhoneがいくら売れたかを1つずつ見て、どの販売カウントが一番高いかを確認する必要がある。
iPhone 13 Pro Maxの販売台数は17台。
iPhone 13 Proの販売台数は9台。
iPhone 13の販売台数は4台。
17、9、4のうち最も大きい数は17。
Answer:iPhone 13 Pro Max。

Table:
| Item Name | SKU | Vendor | Aug 1 Inventory | Sep 1 Inventory | Sale Count |
|---|---|---|---|---|---|
| iPhone 13 Pro Max | MGL83LL/A | Apple | 100 | 80 | 17 |
| iPhone 13 Pro | MLL03LL/A | Apple | 50 | 40 | 9 |
| iPhone 13 | MLKG3LL/A | Apple | 25 | 20 | 4 |
| Samsung Galaxy S22 Ultra | SM-S908U | Samsung | 100 | 80 | 19 |
| Samsung Galaxy S22 Plus | SM-S906U | Samsung | 50 | 40 | 10 |
| Samsung Galaxy S22 | SM-S901U | Samsung | 25 | 20 | 5 |
| Google Pixel 6 Pro | GA01314-US | Google | 100 | 80 | 20 |

Question:
9月1日に最も多くの台数が不明となったサムスンの携帯電話は？
回答 一台ずつ見て、サムスンの商品かどうかを判断する必要があります。
サムスンのアイテムはアイテム名を見なければなりません。
サムスンの品目のみを考慮する。
サムスンのアイテムは、S22 Ultra、S22 Plus、S22です。
それぞれのサムスン製品について、9月1日と8月1日の在庫差を見て、何台売れたはずかを確認します。
そして、その数字と実際の販売台数を比較する必要がある。
販売数フィールドと在庫差の差が最も大きい機種が、最も未カウントの機種です。
Samsung Galaxy S22 Ultraは8月1日に100台在庫があり、9月1日に80台在庫がありました。100から80を引くと20です（100 - 80 = 20）。販売数は19。20から19を引くと1（20 - 19 = 1）。1台が未入荷。
サムスンGalaxy S22 Plusの在庫は8月1日50台、9月1日40台。50から40を引くと10（50 - 40 = 10）。販売数は10。販売数は在庫差と一致し、未計上の台数はない。
サムスンGalaxy S22の在庫は8月1日25個、9月1日20個。25から20を引くと5（25 - 20 = 5）。販売台数は5台、20台マイナス19台は1台。
S22ウルトラのみ、未計上のものがあった。
Answer:Samsung Galaxy S22 Ultra。

Table:
| Item Name | SKU | Vendor | Aug 1 Inventory | Sep 1 Inventory | Sale Count |
|---|---|---|---|---|---|
| iPhone 13 Pro Max | MGL83LL/A | Apple | 100 | 80 | 17 |
| iPhone 13 Pro | MLL03LL/A | Apple | 50 | 40 | 9 |
| iPhone 13 | MLKG3LL/A | Apple | 25 | 20 | 4 |
| Samsung Galaxy S22 Ultra | SM-S908U | Samsung | 100 | 80 | 19 |
| Samsung Galaxy S22 Plus | SM-S906U | Samsung | 50 | 40 | 10 |
| Samsung Galaxy S22 | SM-S901U | Samsung | 25 | 20 | 5 |
| Google Pixel 6 Pro | GA01314-US | Google | 100 | 80 | 20 |

Question:
総売上高が最も多かったベンダーは？
答えてください： ベンダーを一つずつ調べる必要があります。
商品名フィールドからベンダーを推測する必要があります。
テーブルには3つのユニークなベンダーがあります： アップル、サムスン、グーグルです。
各ベンダーについて、各アイテムの販売数を1つずつ見つけ、販売数を合計する必要があります。
アップルの商品は、iPhone 13 Pro Maxが17件、iPhone 13 Proが9件、iPhone 13が4件です。
17 + 9 + 4 = 30. 30台のアップル製携帯電話が売れたことになる。
サムスンの商品は、サムスン・ギャラクシーS22ウルトラで19台、サムスン・ギャラクシーS22プラスで10台、サムスン・ギャラクシーS22で5台。
19 + 10 + 5 = 34. 34台のサムスン製携帯電話が売れた。
グーグル製はグーグルPixel 6 Proで20台。20台のグーグル・スマホが売れた。
アップル30台、サムスン34台、グーグル20台。34が最大の数字で、これはサムスンの販売台数である。
Answer:サムスン


Table:
| Item Name | SKU | Vendor | Aug 1 Inventory | Sep 1 Inventory | Sale Count |
|---|---|---|---|---|---|
| iPhone 13 Pro Max | MGL83LL/A | Apple | 100 | 80 | 17 |
| iPhone 13 Pro | MLL03LL/A | Apple | 50 | 40 | 9 |
| iPhone 13 | MLKG3LL/A | Apple | 25 | 20 | 4 |
| Samsung Galaxy S22 Ultra | SM-S908U | Samsung | 100 | 80 | 19 |
| Samsung Galaxy S22 Plus | SM-S906U | Samsung | 50 | 40 | 10 |
| Samsung Galaxy S22 | SM-S901U | Samsung | 25 | 20 | 5 |
| Google Pixel 6 Pro | GA01314-US | Google | 100 | 80 | 20 |

Question:
最も売れた商品は？
答えてください： ひとつひとつ見ていく必要がありますね。
iPhone 13 Pro Maxは17台売れました。
iPhone 13 Proの販売台数は9台でした。
iPhone13は4台。
サムスン・ギャラクシーS22ウルトラの販売台数は19台。
サムスン・ギャラクシーS22 Plusは10台。
サムスン・ギャラクシーS22は5台。
グーグルPixel 6 Proの販売台数は20台。
販売台数は17台、9台、3台、19台、10台、5台、20台。
20が最大の販売台数で、これはGoogle Pixel 6 Proの販売台数である。
Answer:Google Pixel 6 Pro。

"""

# Prepending the few shot exemplars before the question we want answered.
llm_call = f"{context}\n{few_shot_exemplar}{question}\nAnswer:"
_ = call_llm(model, parameters, llm_call)

さらに2つの質問（読みやすさのモデルの呼び出しを抑制します）：

In [ ]:
# The correct answer is $6.15.
question = """
Table:
| Book Name | Edition | ISBN | Publisher | Aug 1 Amazon Avg New Price | Aug 1 Amazon Avg Used Price | Aug 1 Abebooks Avg New Price | Aug 1 Abebooks Avg Used Price | Sep 1 Amazon Avg New Price | Sep 1 Amazon Avg Used Price | Sep 1 Abebooks Avg New Price | Sep 1 Abebooks Avg Used Price |
|---|---|---|---|---|---|---|---|---|---|---|---|
| Physics for Computer Scientists | 10th | 978-1-118-56906-1 | Pearson Education | $149.99 | $79.99 | $142.94 | $66.94 | $129.99 | $59.99 | $139.94 | $56.94 |
| Fundamentals of Calculus | 8th | 978-0-470-45831-0 | John Wiley & Sons | $139.99 | $99.99 | $137.94 | $87.94 | $129.99 | $79.99 | $129.94 | $76.94 |
| Post-War British Literature | 2nd | 978-0-300-08897-2 | Oxford University Press | $129.99 | $89.99 | $122.94 | $74.94 | $119.99 | $74.99 | $124.94 | $71.94 |
| Modern Religions: An Overview | 3rd | 978-0-19-992545-3 | Oxford University Press | $119.99 | $79.99 | $117.94 | $72.94 | $114.99 | $69.99 | $114.94 | $66.94 |
| The Norton Introduction to Literature | 11th | 978-0-393-45078-1 | W. W. Norton & Company | $129.99 | $89.99 | $122.94 | $74.94 | $119.99 | $74.99 | $124.94 | $71.94 |
| The Norton Anthology of World Literature | 8th | 978-0-393-92855-6 | W. W. Norton & Company | $179.99 | $139.99 | $174.94 | $127.94 | $169.99 | $124.99 | $174.94 | $121.94 |
| The Elements of Style | 5th | 978-0-205-11265-3 | Longman | $119.99 | $79.99 | $117.94 | $72.94 | $114.99 | $69.99 | $114.94 | $66.94 |

Question:
もし8月にアマゾンではなく阿部書房で『エレメンツ・オブ・スタイル』の新刊を3冊購入したら、いくらの節約になるだろうか？
"""

llm_call = f"{context}\n{few_shot_exemplar}{question}\nAnswer:"
print(call_llm(model, parameters, llm_call, show_activity=False))

print("\n\n")

# The correct answer is Physics for Computer Scientists.
question = """
Table:
| Book Name | Edition | ISBN | Publisher | Aug 1 Amazon Avg New Price | Aug 1 Amazon Avg Used Price | Aug 1 Abebooks Avg New Price | Aug 1 Abebooks Avg Used Price | Sep 1 Amazon Avg New Price | Sep 1 Amazon Avg Used Price | Sep 1 Abebooks Avg New Price | Sep 1 Abebooks Avg Used Price |
|---|---|---|---|---|---|---|---|---|---|---|---|
| Physics for Computer Scientists | 10th | 978-1-118-56906-1 | Pearson Education | $149.99 | $79.99 | $142.94 | $66.94 | $129.99 | $59.99 | $139.94 | $56.94 |
| Fundamentals of Calculus | 8th | 978-0-470-45831-0 | John Wiley & Sons | $139.99 | $99.99 | $137.94 | $87.94 | $129.99 | $79.99 | $129.94 | $76.94 |
| Post-War British Literature | 2nd | 978-0-300-08897-2 | Oxford University Press | $129.99 | $89.99 | $122.94 | $74.94 | $119.99 | $74.99 | $124.94 | $71.94 |
| Modern Religions: An Overview | 3rd | 978-0-19-992545-3 | Oxford University Press | $119.99 | $79.99 | $117.94 | $72.94 | $114.99 | $69.99 | $114.94 | $66.94 |
| The Norton Introduction to Literature | 11th | 978-0-393-45078-1 | W. W. Norton & Company | $129.99 | $89.99 | $122.94 | $74.94 | $119.99 | $74.99 | $124.94 | $71.94 |
| The Norton Anthology of World Literature | 8th | 978-0-393-92855-6 | W. W. Norton & Company | $179.99 | $139.99 | $174.94 | $127.94 | $169.99 | $124.99 | $174.94 | $121.94 |
| The Elements of Style | 5th | 978-0-205-11265-3 | Longman | $119.99 | $79.99 | $117.94 | $72.94 | $114.99 | $69.99 | $114.94 | $66.94 |

質問です： アマゾンの新品と中古の価格差が最も大きい本は？
"""

llm_call = f"{context}\n{few_shot_exemplar}{question}\nAnswer:"
print(call_llm(model, parameters, llm_call, show_activity=False))

データ理解のユースケースの場合、データスキーマが事前にデータスキーマを知っている場合は、そのスキーマと一致する必要があります。

一般に、模範的なデータ構造がデータ構造の構造であるほど、LLMが正しく応答する可能性が高くなります。

#### 例：タグ付けデータと構造化されたデータ出力

LLMワークフローの2つの一般的なニーズは、説明からタグまたはカテゴリを生成し、構造化されたデータを出力することです。

この例は両方を行います。タグ付けのパフォーマンスは、特定のタグが最適な理由を通じて、チェーンオブテアの模範とともに改善されます（タグが選択された理由の解釈可能性を提供します）。

さらに、JSONのような一般的なデータ形式であっても、構造化されたデータ出力がどのように見えるかを示すと、パフォーマンスが向上します。

[データソース](https://data.amerigeoss.org/dataset/gsa-json-adc1d)。

In [ ]:
context = """あるデータ・ソースの JSON エントリが与えられた場合、以下のフィールドを持つ JSON を出力し、その理由を説明しなさい：
pii： 真偽値: データセットに個人識別情報が含まれている。
age： データセットが最後に更新されてから何年経っているか。
keywords: キーワード： このデータセットをインデックス化するための新しいキーワード。
最後のテキスト出力はJSONでなければならない。
"""


question = """
{
    "@type" : "dcat:Dataset",
    "description" : "<p>The MDS 3.0 Frequency Report summarizes information for active residents currently in nursing homes. The source of these counts is the residents MDS assessment record. The MDS assessment information for each active nursing home resident is consolidated to create a profile of the most recent standard information for the resident.</p>\n",
    "title" : "MDS 3.0 Frequency Report",
    "accessLevel" : "public",
    "identifier" : "465",
    "license" : "http://opendefinition.org/licenses/odc-odbl/",
    "modified" : "2016-04-05",
    "temporal" : "2012-01-01T00:00:00-05:00/2015-12-31T00:00:00-05:00",
    "contactPoint" : {
      "@type" : "vcard:Contact",
      "fn" : "Health Data Initiative",
      "hasEmail" : "mailto:HealthData@hhs.gov"
    },
    "bureauCode" : [ "009:38" ],
    "keyword" : [ "Activities of Daily Living (ADL)" ],
    "language" : [ "en" ],
    "programCode" : [ "009:000" ],
    "publisher" : {
      "@type" : "org:Organization",
      "name" : "Centers for Medicare & Medicaid Services",
      "subOrganizationOf" : {
        "@type" : "org:Organization",
        "name" : "Department of Health & Human Services"
      }
    }
  }


"""

llm_call = f"{context}\nJSON:{question}\nAnswer:"
_ = call_llm(model, parameters, llm_call)

JSON形式は正しいですが、年齢は間違っており、キーワードは予測されていません。1つの模範を追加すると、正しい応答が得られます。

In [ ]:
one_shot_exemplar = """
JSON:
{

    "@type" : "dcat:Dataset",
    "description" : "The primary purpose of this system of records is to properly pay medical insurance benefits to or on behalf of entitled beneficiaries.",
    "title" : "Medicare Multi-Carrier Claims System",
    "accessLevel" : "restricted public",
    "dataQuality" : true,
    "identifier" : "b6ffafab-1cfd-42dd-b8cb-7a554efaefa7",
    "landingPage" : "http://www.cms.gov/Research-Statistics-Data-and-Systems/Computer-Data-and-Systems/Privacy/Systems-of-Records-Items/09-70-0501-MCS.html",
    "license" : "http://www.usa.gov/publicdomain/label/1.0/",
    "modified" : "2014-09-30",
    "rights" : "Contains personally identifiable information and is subject to the Privacy Act of 1974, as amended at 5 United States Code (U.S.C.) 552a.  Requests should be directed to the appropriate System Manager, identified in the System of Records notice.",
    "primaryITInvestmentUII" : "009-000004256, 009-000004254",
    "systemOfRecords" : "09-70-0501",

    "contactPoint" : {
      "@type" : "vcard:Contact",
      "fn" : "Health Data Initiative",
      "hasEmail" : "mailto:Healthdata@hhs.gov"
    },
    "bureauCode" : [ "009:38" ],
    "keyword" : [ "medicare", "part b", "claims" ],
    "programCode" : [ "009:078" ],
    "theme" : [ "Medicare" ],
    "publisher" : {
      "@type" : "org:Organization",
      "name" : "Centers for Medicare & Medicaid Services",
      "subOrganizationOf" : {
        "@type" : "org:Organization",
        "name" : "Department of Health & Human Services"
      }
    }
  }

Question: rights'タグに「個人を特定できる情報を含む」とあるので、piiはTrueです。
modified' タグは '2014-09-30' です。現在の年は2023年で、2023年から2014年を引くと9なので、年齢は9である。
キーワードを決定するために、データセットを記述するすべてのフィールドを調べる。
そして、そのフィールドの最も顕著で特徴的な点を取り上げて、それをキーワードにする。
すべてのフィールドを見ると、データセットを説明するフィールドは「description」と「title」である。
タイトル」フィールドは「Medicare Multi-Carrier Claims System」である。
title 「フィールドから得られる良いキーワードは、」medicare 「と 」claims "である。
description 「フィールドは、」「この記録システムの主な目的は、権利を有する受益者に、または受益者に代わ って、医療保険給付金を適切に支払うことである」"である。
description 「フィールドからの良いキーワードは 」medical insurance benefits "である。
両フィールドから提案されるキーワードは、「medicare」、「claims」、「medical insurance benefits 」である。
次に 「keyword 」フィールドを検査し、提案されたキーワードがすでに含まれていないことを確認する。
keyword 「フィールドには、」medicare「、」part b「、」claims "というキーワードが含まれている。
提案されたキーワードから、「medicare 」はすでに 「keyword 」フィールドに入っているので、出力されるべきではない。
残るは「クレーム」と「医療保険給付」である。

Output JSON:
{
  "pii" : true,
  "age" : 9,
  "keywords" : ["claims", "medical insurance benefits"]
}
"""

# Prepending the one shot exemplar before the question we want answered.
llm_call = f"{context}{one_shot_exemplar}\nJSON:{question}\nAnswer:"
_ = call_llm(model, parameters, llm_call)

出力は正しいですが、キーワードのオーバーラップの理由がより明確になる可能性があり、これにより、プロンプトがより堅牢になります。これを改善するために考えてから、1つのソリューションの次のセルをご覧ください。

In [ ]:
few_shot_exemplar = """
JSON:
{

    "@type" : "dcat:Dataset",
    "description" : "The primary purpose of this system of records is to properly pay medical insurance benefits to or on behalf of entitled beneficiaries.",
    "title" : "Medicare Multi-Carrier Claims System",
    "accessLevel" : "restricted public",
    "dataQuality" : true,
    "identifier" : "b6ffafab-1cfd-42dd-b8cb-7a554efaefa7",
    "landingPage" : "http://www.cms.gov/Research-Statistics-Data-and-Systems/Computer-Data-and-Systems/Privacy/Systems-of-Records-Items/09-70-0501-MCS.html",
    "license" : "http://www.usa.gov/publicdomain/label/1.0/",
    "modified" : "2014-09-30",
    "rights" : "Contains personally identifiable information and is subject to the Privacy Act of 1974, as amended at 5 United States Code (U.S.C.) 552a.  Requests should be directed to the appropriate System Manager, identified in the System of Records notice.",
    "primaryITInvestmentUII" : "009-000004256, 009-000004254",
    "systemOfRecords" : "09-70-0501",

    "contactPoint" : {
      "@type" : "vcard:Contact",
      "fn" : "Health Data Initiative",
      "hasEmail" : "mailto:Healthdata@hhs.gov"
    },
    "bureauCode" : [ "009:38" ],
    "keyword" : [ "medicare", "part b", "claims" ],
    "programCode" : [ "009:078" ],
    "theme" : [ "Medicare" ],
    "publisher" : {
      "@type" : "org:Organization",
      "name" : "Centers for Medicare & Medicaid Services",
      "subOrganizationOf" : {
        "@type" : "org:Organization",
        "name" : "Department of Health & Human Services"
      }
    }
  }

Answer: rights 「フィールドに 」Contains personally identifiable information "とあるので、piiは本当です。
modified 「フィールドは 」2014-09-30 "です。現在の年は2023年で、2023年から2014年を引くと9なので、年齢は9です。
キーワードを決定するために、データセットを記述するすべてのフィールドを調べます。
そして、そのフィールドの最も顕著で特徴的な点を取り上げて、それをキーワードにする。
すべてのフィールドを見ると、データセットを説明するフィールドは「description」と「title」である。
タイトル」フィールドは「Medicare Multi-Carrier Claims System」である。
title 「フィールドから得られるよいキーワードは、」medicare 「と 」claims "である。
description 「フィールドは、」The primary purpose of this system of records is to properly pay medical insurance benefits to or behalf of entitled beneficiaries "である。
description 「フィールドからのよいキーワードは 」medical insurance benefits "である。
両フィールドから提案されるキーワードは、「medicare」、「claims」、「medical insurance benefits 」である。
次に 「keyword 」フィールドを検査し、提案されたキーワードがすでに含まれていないことを確認する。
keyword 「フィールドには、」medicare「、」part b「、」claims "というキーワードが含まれている。
提案されたキーワードから、「medicare 」はすでに 「keyword 」フィールドに入っているので、出力されるべきではない。
残るは「クレーム」と「医療保険給付金」である。


Output JSON:
{
  "pii" : true,
  "age" : 9,
  "keywords" : ["claims", "medical insurance benefits"]
}


JSON:
{
  "@type": "dcat:Dataset",
  "title": "Data.gov Top 10 Visiting Countries - Archival",
  "description": "This dataset provides top 10 visiting countries by month in Data.gov up to July 2013.",
  "modified": "2016-01-20",
  "accessLevel": "public",
  "identifier": "GSA-32491",
  "dataQuality": true,
  "describedBy": "http://www.data.gov/metric",
  "describedByType": "text/csv",
  "issued": "2013-05-13",
  "license": "https://creativecommons.org/publicdomain/zero/1.0/",
  "spatial": "United States",
  "publisher": {
      "@type": "org:Organization",
      "name": "General Services Administration"
  },
  "accrualPeriodicity": "R/P1M",
  "isPartOf": "GSA-2015-09-14-01",
  "contactPoint": {
      "@type": "vcard:Contact",
      "fn": "Hyon Joo Kim",
      "hasEmail": "mailto:hyon.kim@gsa.gov"
  },
  "distribution": [{
          "@type": "dcat:Distribution",
          "mediaType": "text/csv",
          "format": "text/csv",
          "title": "Data.gov_Top_10_Visiting_Countries.csv",
          "downloadURL": "https://inventory.data.gov/dataset/b0d40da1-a505-476a-a49b-cfc50ea6d9da/resource/0a1a3fb8-a813-4470-b50c-51b7856203be/download/userssharedsdfdata.govtop10visitingcountries.csv"
      }
  ],
  "keyword": ["Countries", "Interactive"],
  "bureauCode": ["023:00"],
  "programCode": ["023:019"],
  "language": ["us-EN"],
  "theme": ["Countries", "Top 10"]
  }

Answer: accessLevel 「フィールドには 」public "とあるので、piiはFalseである。
modified 「フィールドは 」2016-01-20 "である。現在の年は2023年で、2023から16を引くと7なので、年齢は8である。
キーワードを決定するために、データセットを記述するすべてのフィールドを調べる。
そして、そのフィールドの最も顕著で特徴的な点を取り上げて、それをキーワードにする。
すべてのフィールドを見ると、データセットを説明するフィールドは「description」と「title」である。
title 「フィールドは、」Data.gov Top 10 Visiting Countries - Archival "である。
title 「フィールドから得られる良いキーワードは、」data.gov「、」top 10 "である。
description 「フィールドは 」This dataset provides top 10 visiting countries by month in Data.gov up to July 2013 "である。
description 「フィールドからの良いキーワードは、」top 10 「と 」visiting countries "である。
両方のフィールドから提案された良いキーワードは、「data.gov」、「top 10」、「visiting countries 」である。
次に 「keyword 」フィールドを検査し、提案されたキーワードがすでに含まれていないことを確認する。
keyword 「フィールドには、」Countries 「と 」Interactive "というキーワードが含まれています。
提案されたキーワードはどれも 「keyword 」フィールドに含まれていない。
「data.gov「、」top 10「、」visiting countries "はすべて新しいキーワードとして認められる。

Output JSON:
{
  "pii" : false,
  "age" : 9,
  "keywords" : ["data.gov", "top 10", "visiting countries"]
}
"""
llm_call = f"{context}{few_shot_exemplar}\nJSON:{question}\nAnswer:"
_ = call_llm(model, parameters, llm_call)

## zero-shotの思考チェーン（「段階的に考えましょう」）

ゼロショットチェーンの考え方は、LLMコールの最後に「トリガー文」を追加するときです。たとえば、「段階的に考えましょう」、「深呼吸をすることから始めます」、または「解決策：」。これは、迅速かつ簡単なパフォーマンスを向上させる方法であり、さまざまなタスクに柔軟に対応できます（一方、少数の思考の連鎖には、質問に似ている必要があります）。

ただし、ゼロショットの思考チェーンは、ほぼすべての状況で数ショットのパフォーマンスを低下させます。さらに、ゼロショットの思考チェーンでは、LLMを2回呼び出す必要があります - 応答を生成するために、そして再び応答から答えを抽出する必要があります（応答構造を示す模範がないため）。最後に、ゼロショットのチェーンオブテアは、質問に答えるのではなく、質問を再定義する傾向があります。

一般的に、少数のショットチェーンの模範を書くときのインスピレーションを除いて、堅牢なプロンプトをエンジニアリングする場合は、ゼロショットチェーンの考え方は推奨されません。

##思考の連鎖の利点

1. 最小限の努力のための簡単なLLM品質の向上。
1. 問題を解決するための手順を口頭で「話す」ことによって解決できるタスクに適用できます。
1. 解釈可能性。これにより、デバッグが支援され、エンドユーザーの解釈が必要なユースケースが可能になります。
1. 既製のLLMSで動作し、追加のLLMトレーニングやチューニングは必要ありません。
1. 異なるLLM間の堅牢性。考えられたチェーンプロンプトからの最終的な出力は、ドリフトを減らします。

##思考の短所

1. 長いLLMコールと応答によるコストの増加。
1. 推論時間が遅い。
1. 幻覚はまだ可能です。

##思考のチェーンベストプラクティス

これらの推奨事項は、現在の理解を反映しており、LLMはすべて急速に変化しています。これのいくつかは、特定のコーナーケースとLLMアーキテクチャでは間違っている可能性があります。

これらのベストプラクティスの例外を見つけた場合は、GitHubの問題を提出することを検討してください。

###重要なベストプラクティス

思考の連鎖から良いパフォーマンスを得るには、これらのベストプラクティスに従う必要があります。

1. 小さなLLMを**使用しない**でください。
  * 理想的には、少なくとも15BパラメーターのLLMを使用します。
  * 蒸留や改良されたLLMアーキテクチャのような技術が、最終的にこのアドバイスを変えることを期待したい。
1. 思考の連鎖の前に答えを書くのではなく、思考の連鎖の後に答えを書く。
1. [温度](https://cloud.google.com/vertex-ai/docs/generative-ai/start/quickstarts/api-quickstart#try_text_text_prompts)を0に設定する。
1. ワンショットやゼロショットだけでなく、数ショットの思考連鎖を使うこと。
1. ステップ・バイ・ステップで推論を話すときに言うことをすべて盛り込んだ模範解答を書くこと。
  * 思考の連鎖には自然言語による推論が必要だ。
  * 自然言語による推論の代わりに数式を使わない。自然言語を補足するために方程式を加えるのは構わない。
1. 思考の連鎖が幻覚を止めると決めつけてはいけない。
  * 思考の連鎖はLLMの推論能力を向上させるが、LLMが事実をでっち上げることを止めるわけではない。

###追加のベストプラクティス

思考の連鎖から最大限に活用するためのより多くのヒント。

1. Few-Shotプロンプト作成のDo's and Don'ts
Don't 過度にFew-Shot事例の順序にこだわる必要はありません。パフォーマンスに変化はないでしょう。

  * 分類タスクは例外です。同じクラスの事例を連続して複数提示しないようにしましょう。

1. Do Chain-of-Thoughtプロンプトが失敗する箇所を分析し、よくある失敗に対処するためのFew-Shot事例を作成しましょう。

1. Don't 最初から6つ以上のFew-Shot事例を作成する必要はありません。タスクによっては、それ以上の事例が役立つ場合もありますが、多くの場合は不要です。

1. Do 複数のプロンプトエンジニアにそれぞれ最適なプロンプトを作成してもらいましょう。

  * 例えば、3つのタスクがあり、3人のプロンプトエンジニアがいる場合、各エンジニアが1つのタスクに集中するよりも、全員が3つのタスクのプロンプトを作成する方が良い結果が得られるでしょう。

1. Don't タスクに必要な推論ステップが1つか2つの場合、Chain-of-Thoughtで結果が改善するとは期待しないでください。

1. Don't 事例とタスクの推論ステップ数を厳密に一致させることに気を使いすぎる必要はありません。

  * 推論のスタイルや構造を一致させる方が重要です。
  * ステップ数を一致させることができればパフォーマンス上の利点がありますが、できなくてもChain-of-Thoughtは依然としてパフォーマンス向上に貢献します。

1. Do LLMをチューニングする際にChain-of-Thoughtを追加しましょう。

  * LLMに質問と回答からChain-of-Thoughtによる推論を生成させ、その推論をチューニングデータの回答に追加することができます。
  * プロンプトとチューニングは二者択一ではありません。チューニングデータの入力が適切に設計されたプロンプトを含む場合、最適なチューニングモデルパフォーマンスが得られます。

1. Do データ分布に一致する事例を含めましょう。

  * 例えば、データがクラスA 80%、クラスB 20%で、5つのFew-Shot事例を作成する場合、4つの事例はクラスA、1つの事例はクラスBにするべきです。
  * 分類タスクでは事例の順序が重要になることもありますが、クラス分布を一致させることで順序に対する頑健性が高まります。
  * 連続して同じクラスの事例を複数提示しないように注意しましょう。

##自己整合性

自己整合性は、一連の思考プロンプトのパフォーマンスを改善するための手法です。同じLLMコールを複数回呼び出して、最も一般的な答えを出します。

これは、温度= 0で思考の連鎖を使用するためのルールを「破る」ことを意味します。

自己整合性の背後にある直観は次のとおりです。
1.同一のLLM呼び出しに対する複数の応答は、応答のさまざまな推論パスを意味します。
1.誤った推論パスは、異なる誤った回答につながります。
1.正しい推論パスは同じ正解につながります。
1.いくつかの正解と多くの誤った答えしか得られないかもしれませんが、正解は一意の誤った答えよりも一般的です。

自己整合性を試してみましょう。まず、温度0でこの次のLLM呼び出しを実行して、誤った応答を生成します。

In [ ]:
# The answer is 1300 + 100 (maintenance) + 75 (upgrade) = 1475.
question = """工場の基本生産性は1日100ユニットである。
すべての工場が基本生産性を有しているわけではない。
工場がアップグレードされているときは、ベースラインの生産性の 25％である。
工場がメンテナンスを受けているときは、ベースラインの 50％である。
工場が労働活動中の場合、何も生産しない。
メガコープには合計 19 の工場がある。
3 工場がアップグレード中。
2 工場がメンテナンス中。
1 工場が労働活動中である。
メガコーポは1日に何ユニットを生産するか？"""

context = """完全な計算と推論を示す質問に答える。
例題のパターンに従いましょう。
"""

one_shot_exemplar = """Q: 通常のテニスボールは5個入ります。
大きなテニスボール缶は、通常のテニスボール缶の200％のボールを入れることができます。
小さなテニスボール缶には、通常のテニスボール缶の40%のボールが入ります。
コレクタブルテニスボール缶には、テニスボールは入りません。
ロジャーは10個のテニスボール缶を持っている。
3つの缶は大きい缶である。
4缶は小さい缶。
1缶はコレクション缶である。
ロジャーは何個のテニスボールを持っていますか？
A: 通常のテニスボール缶の数を求める必要がある。
ロジャーは、10(合計)-3(大きい)-4(小さい)-1(収集可能)=2個のレギュラー缶を持っています。
大きなテニスボール缶には、5個の200％＝10個のテニスボールが入ります。
小さなテニスボール缶には、5の40％＝2個のテニスボールが入ります。
次に、それぞれの缶の種類から何個のボールが出るかを数える。
大きい缶3個は、3 * 10 = 30個のテニスボール。
小さい缶4個は、2 * 4 = 8個のテニスボール。
レギュラー缶2個は、2 * 5 = 10テニスボール
コレクタブル缶1個はテニスボール0個
答えを出すには、それぞれの缶の種類のボールの数を足します。
ロジャーは30個（大）＋8個（小）＋10個（レギュラー）＋0個（コレクタブル）＝48個のボールを持っている。
答えは48個です。

Q: """

llm_call = f"{context}\n{one_shot_exemplar}{question}\nA:"
_ = call_llm(model, parameters, llm_call)

次に、「温度」を.7に上げ、高い「TOP_P」と「TOP_K」値を使用して異なる応答を生成します。

次のセルを数回実行し、答えがどのように変化するかに注意してください。

In [ ]:
sc_parameters = {
    "temperature": .7,
    "max_output_tokens": 512,
    "top_p": 1,
    "top_k": 40
}

_ = call_llm(model, sc_parameters, llm_call)

上記のコードを再実行すると、さまざまな推論と回答が表示されます。

次に、多くの応答をループして生成し、回答を抽出し、回答を最も一般的で最も一般的ではないように出力します。

これには数分かかります。実行中は、さまざまな推論と回答に注意してください。

In [ ]:
from collections import Counter  # Easy counting of most common responses.
sc_runs = 40
responses = [None] * sc_runs
answers = [None] * sc_runs

for i in range(0, sc_runs):
  print(f"Response {i}...")
  responses[i] = call_llm(model,
                          sc_parameters,
                          llm_call,
                          # Turn off printing LLM calls/responses.
                          show_activity=False)
  # If the response doesn't contain 'The answer is', the split fails.
  # The split also fails if the answer contains a decimal or comma.
  try:
    answers[i] = responses[i].split("The answer is")[1].split(".")[0].strip()
  except Exception as e:
    answers[i] = "NA"
  print(responses[i])
print("最も一般的なものから最も一般的でないものまで、答えとカウント：")
print(Counter(answers).most_common())

上記のセルからの最後の出力は、異なる回答のカウントです。正解（1475）は、最も一般的な答えとして戻ってくるはずです。

LLMの呼び出しが多いほど、最も一般的な答えは正しい答えです。

また、結果をプロットして、回答の分布を視覚化することもできます。

In [ ]:
# Thanks to Hans-Christian Fuchs for this.
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.bar(Counter(answers).keys(), Counter(answers).values())
ax.tick_params(axis='x', rotation=55)
plt.show()

###自己整合性の利点

1. 低エフォルトのパフォーマンスブースト。
1. 思考の模範を助けます。
1. 異なるLLMにわたる迅速な堅牢性の増加。
1. 回答分布に基づいて、擬似「信頼」推定値を提供します。
1. 単一の正解なしで問題に対して「平均」回答を使用する機会。

###自己整合性の欠点

1. コストの増加。
1. 推論時間の遅いおよび/またはスループットの削減。

### 自己整合のベストプラクティス

1. Do temperature=.7、top_k=40、top_p=1、そして10個のレスポンスを初期設定として使用しましょう。

  * ユースケースによって異なる値が必要になる場合があるため、そこから実験を行いましょう。
  * 本番環境で使用する最適な値を見つけるために、ハイパーパラメータ探索を実施しましょう。
  * LLMパラメータよりもレスポンス数を探索する方がはるかに価値がある可能性が高いことに注意してください。また、LLMパラメータを試す場合でも、それらを大幅に減らすことは通常価値がありません。

1. Do 初期のプロンプトエンジニアリングの試みが失敗した場合、早めに自己整合性を試しましょう。

  * 自己整合性は、連鎖思考プロンプトのエンジニアリングを続けるよりも、パフォーマンスを向上させる可能性が高いです。

1. Don't コストとレイテンシの影響を無視しないでください。

1. Do 実行時間を短縮するためにLLM呼び出しを並列化しましょう。

  * 自己整合性ユースケースに必要なLLMスループットとレイテンシの評価を後回しにしないでください。

1. Do レスポンス分布を創造的な方法で使用しましょう。例えば：

  * Xパーセント未満の回答しか一致しない場合、その質問にフラグを立てて人間のレビューに回しましょう。
  * 複数の要約を生成し、テキスト類似性指標を使用して、どの生成された要約が最も「平均的」かを特定しましょう。

1. Do 自己整合性をFew-Shot事例の作成やプロンプトのデバッグに活用しましょう。

# パート2：アクション、検索、ツールの使用

LLMは、カラスのように、ツールを使用するのに熟達しています。

<img src = "https://raw.githubusercontent.com/GoogleCloudPlatform/specialized-training-content/184be57c9ff4de18e20f3fdfbe1ef7fb35ce023f/courses/generative_ai/app_dev_llm/images/3-crow.png">

##幻覚、接地、ツール/アクション/検索/ぼろきれ
<a name="rag"> </a>

LLMは信頼できる事実源ではありません。LLM応答に正しい事実が含まれている場合、LLMのパラメーターが実際にエンコードするものの緊急効果です。単語間の確率的関係です。

事実の正確性が重要な場合、これらの確率的関係に依存することは危険です。

また、LLMSは、最新情報について迅速または安価に再訓練することもできません。また、再訓練が可能性がある場合でも、壊滅的な忘却は、トレーニングデータセットが増加するにつれて、古い情報の新しいエラーにつながる可能性があります。

LLM応答が事実上正しくない場合、それはしばしば「幻覚」と呼ばれますが、より正確には[妄想](https://en.wikipedia.org/wiki/delusion)です。

幻覚は非専門家によって見逃される可能性があります。LLM応答は、生成されたテキストが文法的に正確で、よく形成され、トーンに自信がある場合でも、事実上正しくありません。

このLLM呼び出しが出力するものを参照してください。

In [ ]:
question = "ドイツの首相は誰ですか？"
_ = call_llm(model, parameters, question)

現在のモデルは正しく反応する可能性がありますが、2023年8月、メルケル首相が辞任してからほぼ2年後、これが応答でした。
<img src = "https://raw.githubusercontent.com/GoogleCloudPlatform/specialized-training-content/184be57c9ff4de18e20f3fdfbe1ef7fb35ce023f/courses/generative_ai/app_dev_llm/images/6-hallucinate.png">

幻覚を管理する最良の方法は、LLMを正確で最新の外部データソースに接続することです。

「接地」とは、外部情報を使用して幻覚を管理することです。「グラウンド」する1つの方法は、挿入された情報に基づいて応答を基にするための手順とともに、外部情報をLLMコールに挿入することです。

「検索拡張生成」または「ラグ」は、LLMが外部知識を使用していると言う一般的な方法です。それは異なることを意味する可能性があります：
1. 外部検索システムは、ユーザークエリを入力として取得し、情報を出力し、LLMコールのユーザークエリと組み合わせます。（たとえば、クエリの埋め込みをドキュメントの埋め込みと比較し、LLMコールに最も近いドキュメントを挿入します）。[コードサンプル](https://github.com/googlecloudplatform/generative-ai/blob/main/language/use-cases/document-qa/question_answering_documents_langchain_matching_engine.ipynb)。

1. ユーザーのクエリに基づいて外部情報システムへの検索コールを策定する手順でLLMを呼び出し、ユーザーのクエリと取得情報を組み合わせてLLMに別の呼び出しを行います。

3. 結合したオーダーメイドレトリバーとジェネレーターディープラーニングモデルを一緒に訓練/調整しました（元の[RAG Paper](https://arxiv.org/pdf/2005.11401.pdf)の焦点）。

このノートブックは# 2に焦点を当てており、言語「ツール」/「ツール使用」を使用して、LLMを指示するように外部システムを使用するように説明し、あいまいな用語のぼろを避けます。パート3の後半では、「アクション」と「演技」を使用して、Reactが議論される方法と一致します。

## LLMツールの使用方法

LLMツールの使用の基本パターンは次のとおりです。
1. 以下を説明する最初のLLMコールを作成します。
* I：完了したいタスク。
* II：外部システム。
* III：外部システムへの呼び出しを策定する方法。
2. LLMによって生成された応答を使用して外部システムを呼び出します。
3.外部システムからの応答を含む2番目のLLMコールを作成し、LLMが外部システムからの応答を使用して元のタスクを完了するように指示します。

私たちのLLMシステムが、上記の首相の例のような事実ベースの質問に答えることになっている場合：
1. 最初のLLMコールは、LLMを指示して、知識ベースの検索クエリを生成します。
2. LLMの応答は、知識ベースを照会するために使用され、クエリの結果がキャプチャされます。
3. 2番目のLLMコールには、ナレッジベースクエリ、元の質問、およびLLMがナレッジベースクエリの結果を使用して質問に答えるための指示の結果が含まれます。

LLMのツールは、データベース、Web検索、ドキュメント検索システムなど、多くのものになる可能性があります。LLMシステムの一部は、LLMを外部情報ソースと統合するコードです。

このノートブックでは、ウィキペディアを外部情報ソースとして使用し、基本的なLLMシステムを構築して、事実ベースの質問に答えます。私たちのLLMシステムは次のとおりです。
1. LLMを呼び出して、ウィキペディア検索クエリを生成します。
1. ウィキペディアAPIを呼び出して、クエリの結果を取得します。
1. Wikipedia API応答と元の質問を使用して、LLMをもう一度呼び出します。

このノートブックの範囲を超えて、LLMSは、複数のツールを説明するInstandinosで呼び出すことができます。LLMはどちらもツールを選択し、ツールへの呼び出しを策定します。また、LLMツールは読み取り専用である必要はありません。ツールを使用して外部システムと対話できます（ただし、倫理と公平性への影響を考慮してください。Tは、 *幻覚は概要を尋ねたいと思っていますが、自動化された紙のグレーディングに影響を与えるように、誰かの人生に影響を与える可能性があるという決定を下すと、壊滅的です。誰かの人生）。

##サンプルツール

以下の関数はクエリを取り、クエリのトップウィキペディア記事マッチを返し、記事の最初の `return_chars`文字を取得します。

このツールは教育目的であり、やや制限されています。リストやサイドバーにアクセスすることはできず、提案をうまく処理せず、ウィキペディアの記事内での検索をサポートせず、常に結果を返すとは限りません。

In [ ]:
import wikipedia
def wiki_tool(query, return_chars = 1000):
  try:
    page = wikipedia.page(query, auto_suggest=False, redirect=True).content
  # If no exact match, take Wikipedia's auto-suggestion.
  except wikipedia.exceptions.PageError as e:
    page = wikipedia.page(query, auto_suggest=True, redirect=True).content
  snippet = page[0:return_chars]
  return snippet

ツールを試してください：

In [ ]:
wiki_tool("ドイツ首相")

##チェーンLLMはツールの使用を求めています

基本的な2段階のツール使用LLMチェーンには、ここで段階的に分類されているいくつかのピースが含まれています。

この例でモデルを（2023年10月の時点で）、あいまいなミュージシャンについての質問に電話すると、誤った答えが幻覚を起こします。

In [ ]:
question = "アルバム「Somebody in the Snow」をリリースしたミュージシャンは？"
_ = call_llm(model, parameters, question)

###ステップ1：ツールを使用するためのLLMの指示を提供する

LLMに、タスクとツールの使用方法の両方の指示を提供する必要があります。

LLM呼び出しのこの「命令」部分は、「コンテキスト」または「条件」（「コンディショニング」、「コンディショニングプロンプト」）のバリエーションと呼ばれることがあります。

In [ ]:
context = """ウィキペディアの検索を使って質問に答えなさい。
各質問の後に、ウィキペディア検索の後に'<STOP>'を書く。
ウィキペディア検索は、最も関連性の高いコンテンツを検索するために使用されます。
ウィキペディアの記事の一部が次のLLMコールに送られます。
ウィキペディアの記事のテキストを使って質問に答えてください。"""

###ステップ2：模範を提供します

LLMには、ツールを使用してタスクを完了する方法を示す模範が必要です。

この例には、1ショットの模範的なものしかありません。

この模範のウィキペディアの記事のテキストは、2023年8月に「wiki_tool（ "chancellor ofドイツ"）を実行しています。

注：将来の再試行の後、LLMは外部ツールなしでこの質問に正しく答えます。しかし、このワンショットの模範は、ウィキペディア検索のパターン、応答、および応答に基づいた回答を示すため、まだ機能します。

In [ ]:
exemplar = """質問 ドイツの首相は誰ですか？
ウィキペディアで検索： ドイツ首相<STOP
ウィキペディアの記事 ドイツ首相（ドイツれんぽうしゅしょう）、通称ドイツ首相は、ドイツ連邦共和国の首相であり、ドイツ連邦政府の首班である。首相は連邦内閣の最高責任者であり、行政府の長である。現在の首相はSPDのオラフ・ショルツで、アンゲラ・メルケルの後任として2021年12月に選出された。現在の首相はSPDのオラフ・ショルツで、アンゲラ・メルケルの後任として2021年12月に選出された。SPDが同盟90/緑の党、FDPと連立協定を結んだ後に選出された。この称号は、ドイツ語圏ヨーロッパのいくつかの州で使われたこともある。近代的な大法官職が確立されたのは、神聖ローマ帝国時代からである。
答えは？オラフ・ショルツ"""

###ステップ3：LLMチェーンで最初の呼び出しを行う

私たちのコンテキストと模範を質問と組み合わせて、Wikipediaの検索クエリを回答として要求するLLMに電話をかけます。

In [ ]:
step_one_call = f"""{context}

{exemplar}

Question: {question}
Wikipedia Search:"""
step_one_response = call_llm(model, parameters, step_one_call)

###ステップ4：LLMの応答を使用してツールをクエリします

注LLM応答には、Wikipedia検索クエリ以上のものが含まれています。

LLMSは、LLMコールのトークンと以前に予測されたトークンのトークンに基づいて、次のトークンを何度も繰り返し予測することで機能します。これは、LLMが過剰なテキストを生成することを意味します。ウィキペディア検索クエリの後に停止することはわかりません。

ウィキペディアの検索クエリを超えたすべてのものはごみです。余分なテキストは「<stop> `signifierを使用して破棄されますが、これはラインブレークでも実行できます。

生産システムでは、このようなLLMコールを行うときに応答サイズを制限することにより、コストを制御することが重要です。

次の関数は、最初のチェーンステップからLLM応答を取得し、ウィキペディアクエリを返します。

In [ ]:
def get_wiki_query (llm_response, stop_text = "<STOP>"):
  # Assumes the query is in the first line.
  first_line = llm_response.splitlines()[0]
  query = first_line.split(stop_text)[0]
  return query.strip() # Remove leading and trailing whitespace.

以前のLLMコールからの応答でこの関数を使用してクエリを抽出し、「wiki_tool」を使用してウィキペディアを検索します。

In [ ]:
wiki_query = get_wiki_query(step_one_response)
print(f"Tool Query: {wiki_query}")
wiki_text = wiki_tool(wiki_query)
print(f"Wikipedia Snippet: {wiki_text}")

###ステップ5：ツール応答を使用して、LLMチェーンで2回目の呼び出しを行う

次に、ツールから出力を取得し、2番目のLLMコールを作成して質問に答えます。

LLMツールの使用は、一般に、以前の呼び出しと応答の履歴を維持しています。チェーン内の2番目の呼び出しを作成するには：
1. チェーン内の最初のLLMコールから始めます。
1. 以前に生成されたウィキペディアクエリを追加します。
1. ウィキペディアの検索結果を追加します。

これが私たちの最初の呼び出しがどのように見えるかを思い出させます：

In [ ]:
print(step_one_call)

この最初のLLMコールは、最初のLLM応答からのクエリと、ウィキペディアツールからの出力と、模範と一致する構造と組み合わされます。

In [ ]:
step_two_call = f"""{step_one_call} {wiki_query}
Wikipedia Article: {wiki_text}
Answer: """
step_two_response = call_llm(model, parameters, step_two_call)

##すべてのステップをまとめる

下のこのコードスニペットは、上記のすべての手順、従属パッケージ、および従属関数を2段階のツール使用LLMチェーンを管理する単一の関数に収集します。

適切なパッケージをインストールして認証されたと仮定して、このコードを独自のプロジェクトにコピーして貼り付けることができます。

In [ ]:
import wikipedia

def call_llm(model, parameters, llm_call, show_activity = True):
  # Wraps an LLM call to Vertex, optionally displaying the call and response.
  response = model.predict(llm_call, **parameters).text

  if show_activity:
    BOLD = "\033[1m"
    UNFORMAT = "\033[0m\x1B[0m"
    print(f"{BOLD}The call to the LLM:{UNFORMAT}\n{llm_call}\n")
    print(f"{BOLD}The response:{UNFORMAT}")
    print(response)

  return response  # Return to `_` if not needed.


def wiki_tool(query, return_chars = 1000):
  try:
    page = wikipedia.page(query, auto_suggest=False, redirect=True).content
  # If no exact match, take Wikipedia's suggestion.
  except wikipedia.exceptions.PageError as e:
    page = wikipedia.page(query, auto_suggest=True, redirect=True).content
  snippet = page[0:return_chars]
  return snippet


def get_wiki_query (llm_response, stop_text = "<STOP>"):
  # Extract the wikipedia query from the LLM response.
  # Assumes the query is in the first line.
  first_line = llm_response.splitlines()[0]
  query = first_line.split(stop_text)[0]
  return query.strip() # Remove leading and trailing whitespace


def wiki_tool_chain(model,
                    parameters,
                    context,
                    exemplar,
                    question,
                    show_activity=False):
  # Answer a query using wikipedia by calling an LLM.
  step_one_call = (
      f"{context}\n\n{exemplar}\n\nQuestion: {question}\nWikipedia Search:"
  )
  if show_activity:
    print("\033[1mMaking the first LLM call...\033[0m\x1B[0m")
  step_one_response = call_llm(model, parameters, step_one_call, show_activity)
  wiki_query = get_wiki_query(step_one_response)
  wiki_text = wiki_tool(wiki_query)

  step_two_call = (
      f"{step_one_call} {wiki_query}\nWikipedia Article: {wiki_text}\nAnswer: "
  )
  if show_activity:
    print("\033[1mMaking the second LLM call...\033[0m\x1B[0m")
  step_two_response = call_llm(model, parameters, step_two_call, show_activity)

  return step_two_response

上記のコードを使用する例：

In [ ]:
import vertexai
from vertexai.language_models import TextGenerationModel

# Outside this notebook, set PROJECT_ID, LOCATION, and MODEL_NAME.
# When running in the notebook, these are set in part 0.
vertexai.init(project=PROJECT_ID, location=LOCATION)
# These settings control how deterministic the LLM response is.
parameters = {
    "temperature": 0,
    "max_output_tokens": 256,
    "top_p": 0.8,
    "top_k": 40
}
model = TextGenerationModel.from_pretrained(MODEL_NAME)

context = """ウィキペディアの検索を使って質問に答えなさい。
各質問の後に、ウィキペディア検索の後に'<STOP>'を書く。
ウィキペディア検索は、最も関連性の高いコンテンツを検索するために使用されます。
ウィキペディアの記事の一部が次のLLMの呼び出しに送られます。
ウィキペディア記事のテキストを使って質問に答えてください。"""

exemplar = """質問:ドイツの首相は誰ですか?
ウィキペディア検索: ドイツ首相<STOP>
ウィキペディアの記事: ドイツ首相、正式にはドイツ連邦共和国連邦首相は、ドイツ連邦政府の長であり、戦時中のドイツ軍の最高司令官です。首相は連邦内閣の最高責任者であり、行政府の長です。首相は連邦大統領の提案に基づき、議論を経ずに連邦議会によって選出される（ドイツ憲法第63条）。現在の首相はアンゲラ・メルケル首相の後任として2021年12月に選出されたSPDのオラフ・ショルツ氏である。同氏は、SPD が同盟 90/緑の党および FDP と連立協定を結んだ後に選出されました。\n\n\n== 首相の職の歴史 ==\n首相の職には長い歴史があり、その起源は聖公会にまで遡ります。ローマ帝国、当時ドイツの大宰相の職は通常マインツ大司教が務めていた。このタイトルは、ドイツ語圏のヨーロッパのいくつかの州で時々使用されました。近代的な首相官邸は、
答え：オラフ・ショルツ"""

answer = wiki_tool_chain(model,
                         parameters,
                         context,
                         exemplar,
                         "アルバム「Somebody in the Snow」をリリースしたミュージシャンは？",
                         show_activity = False)
print(answer)


`show_activity = true`を使用して、LLMコールの内訳を確認します。

In [ ]:
wiki_tool_chain(model,
                parameters,
                context,
                exemplar,
                "アルバム「Somebody in the Snow」をリリースしたミュージシャンは？",
                show_activity = True)

「質問」を変更して実験してみてください。`show_activity = true`を保持して、LLMチェーンの2つのステップを確認します。

これは多くの質問ではうまくいきません。上記のように、私たちのツールはあまり良くなく、いくつかの質問で完全に失敗します。

ツールの使用ベストプラクティスは[パート3で詳細](https://github.com/GoogleCloudPlatform/specialized-training-content/blob/184be57c9ff4de18e20f3fdfbe1ef7fb35ce023f/courses/generative_ai/app_dev_llm/#react-tools)。

# パート3：反応（推論 +演技）プロンプト

React（推論 +アクション）は、外部システムと対話することにより、複雑なタスクを介して、思考とツールの使用のチェーンとツールの使用を組み合わせて合計します。

Reactスタイルのプロンプトは、現在（2023年秋）ほとんどのプロンプト駆動型LLMタスクの最先端です。LLMまたはLLMベースのチャットボットまたはシステムが外部システムと対話するプラグインまたは拡張機能を使用すると、Reactスタイルのシステムを使用しています。一般に、最新の知識を反映するLLMシステムは、Hood-The-Hoodの下で反応スタイルの機能を使用して目に見えません。

外部システムと対話しようとするLLM：

<img src = "https://raw.githubusercontent.com/GoogleCloudPlatform/specialized-training-content/184be57c9ff4de18e20f3fdfbe1ef7fb35ce023f/courses/generative_ai/app_dev_llm/images/4-robot.png">

##反応の基本

反応チェーンには通常、3つのインターリーブパーツがあります。
-  **思考**：思考の連鎖のように、これらは最終出力に向けて進歩するため、LLMによって生成されるウェイポイント、計画、推論などです。
-  **アクション**：LLMが生成したコマンド、呼び出し、または外部システムにアクセスする手順。外部システムは、情報を提供するツールである可能性がありますが、より一般的なものである可能性があります（つまり、アクションは外部システムの状態を観察または変更します）。
-  **観測**：外部システムからの応答、フィードバック、結果など。LLMコールに挿入されて次の思考を生成します。

これらの3つのステップは、LLMがタスクを完了するまで繰り返されます。

考え方のプロンプトと同様に、この繰り返されるサイクルは「内部の独白」または「内部スピーチ」を形成しますが、行動する決定を重要な追加と、単なる推論を超えて行動からフィードバックします。

###反応チェーンはどのように見えますか

ReactチェーンでLLMコールを分解する前に、完全なReactチェーンがどのように見えるかを確認するのに役立ちます。

このチェーンでのアクションはウィキペディアの検索であり、観察はウィキペディアの記事からのスニペットです。

LLMへの元の呼び出しは次のとおりです。
「質問：誰が最初に生まれたのか、ロナルド・リーガンまたはジェラルド・フォード？」（今のところ、指示、模範などを無視して）。

完成したReactチェーンはこのように見えます。完全な観察結果を読むために右にスクロールします。

```
質問：最初に生まれたのは誰ですか、ロナルドレーガンまたはジェラルドフォード？
考え1：ロナルド・レーガンを調べて、彼がいつ生まれたのかを見る必要があります。
アクション1：ロナルドレーガン<Stop>
観察1：ロナルド・ウィルソン・レーガン（1911年2月6日 -  2004年6月5日）は、1981年から1989年まで米国の第40代大統領を務めたアメリカの政治家および俳優でした。そして、最初の離婚した大統領。レーガンはイリノイ州タンピコで生まれ、イリノイ州ディクソンで育ちました。彼はユーレカ大学で教育を受け、そこで経済学と社会学を学びました。卒業後、レーガンはカリフォルニアに移り、そこでラジオスポーツアナウンサーになりました。彼は後に演技に引っ越し、50以上の映画に出演しました。レーガンは、1947年から1952年までスクリーン俳優ギルドの社長を務めました。
考え2：ロナルド・レーガンは1911年に生まれました。ジェラルド・フォードを調べて、彼がいつ生まれたのかを見る必要があります。
アクション2：ジェラルドフォード<Stop>
観察2：ジェラルド・ルドルフ・フォード・ジュニア（jerr-əld;生まれたレスリー・リンチ・キング・ジュニア、1913年7月14日 -  2006年12月26日）は、1974年から1977年に米国第38代大統領を務めたアメリカの政治家でした。以前は、1965年から1973年まで米国下院で共和党の指導者を務め、スピロアグニューの辞任の後、リチャードニクソン大統領によって40番目の副大統領に任命されました。フォードは、ニクソンが1974年に辞任したときに大統領職に成功しましたが、1976年に完全な任期に選挙で敗北しました。フォードは、大統領または副大統領の選挙を勝ち取らずに米国大統領になった唯一の人物です。フォードはネブラスカ州オマハで生まれ、ミシガン州グランドラピッズで育ちました。彼はミシガン大学に通い、そこで学校のフットボールチームでプレーしてから最終的にイェールロースクールに通いました。その後、彼は1942年から1946年まで米国海軍保護区に勤務しました。フォードは1949年にミシガン州5の米国代表として政治的キャリアを始めました。
考え3：ジェラルドフォードは1913年に生まれました。1911年は1913年以前です。回答[ロナルドレーガン]
```

###反応チェーンを分解します

上記の例の反応チェーンは、3つのLLM呼び出しから構築されています。

注このセクションの回答は、パート2ツールでのディスカッションで追加のテキストがどのように削除されたかと同様に、余分な予測されたテキストを削除されています。

**電話1：**
```
質問：最初に生まれたのは誰ですか、ロナルドレーガンまたはジェラルドフォード？
考え1：
```
**応答1：**
```
ロナルド・レーガンを調べて、彼がいつ生まれたのかを見る必要があります。
アクション1：ロナルドレーガン<Stop>
```

最初のLLMコールは次のとおりです。
1.以前のLLMコールプラス
1.前のコールプラスへのLLM応答
1.ウィキペディアルックアップ結果プラス
1.「思考#：」

**コール2：**

コール2は、連結コール1 +応答1 +ウィキペディアルックアップの結果（観察中） +「思考2：」によって作成されます。
```
質問：最初に生まれたのは誰ですか、ロナルドレーガンまたはジェラルドフォード？
考え1：ロナルド・レーガンを調べて、彼がいつ生まれたのかを見る必要があります。
アクション1：ロナルドレーガン<Stop>
観察1：ロナルド・ウィルソン・レーガン（1911年2月6日 -  2004年6月5日）は、1981年から1989年まで米国の第40代大統領を務めたアメリカの政治家および俳優でした。そして、最初の離婚した大統領。レーガンはイリノイ州タンピコで生まれ、イリノイ州ディクソンで育ちました。彼はユーレカ大学で教育を受け、そこで経済学と社会学を学びました。卒業後、レーガンはカリフォルニアに移り、そこでラジオスポーツアナウンサーになりました。彼は後に演技に引っ越し、50以上の映画に出演しました。レーガンは、1947年から1952年までスクリーン俳優ギルドの社長を務めました。
考え2：
```

**応答2：**
```
ロナルド・レーガンは1911年に生まれました。ジェラルド・フォードを調べて、彼がいつ生まれたのかを見る必要があります。
アクション2：ジェラルドフォード<Stop>
```

**電話3：**

コール2と同じように、Wikipedia Lookup + "Thought 3："の結果、コール2 +応答2 +を連結してコール3を作成します。

```
質問：最初に生まれたのは誰ですか、ロナルドレーガンまたはジェラルドフォード？
考え1：ロナルド・レーガンを調べて、彼がいつ生まれたのかを見る必要があります。
アクション1：ロナルドレーガン<Stop>
観察1：ロナルド・ウィルソン・レーガン（1911年2月6日 -  2004年6月5日）は、1981年から1989年まで米国の第40代大統領を務めたアメリカの政治家および俳優でした。そして、最初の離婚した大統領。レーガンはイリノイ州タンピコで生まれ、イリノイ州ディクソンで育ちました。彼はユーレカ大学で教育を受け、そこで経済学と社会学を学びました。卒業後、レーガンはカリフォルニアに移り、そこでラジオスポーツアナウンサーになりました。彼は後に演技に引っ越し、50以上の映画に出演しました。レーガンは、1947年から1952年までスクリーン俳優ギルドの社長を務めました。
考え2：ロナルド・レーガンは1911年に生まれました。ジェラルド・フォードを調べて、彼がいつ生まれたのかを見る必要があります。
アクション2：ジェラルドフォード<Stop>
観察2：ジェラルド・ルドルフ・フォード・ジュニア（jerr-əld;生まれたレスリー・リンチ・キング・ジュニア、1913年7月14日 -  2006年12月26日）は、1974年から1977年に米国第38代大統領を務めたアメリカの政治家でした。以前は、1965年から1973年まで米国下院で共和党の指導者を務め、スピロアグニューの辞任の後、リチャードニクソン大統領によって40番目の副大統領に任命されました。フォードは、ニクソンが1974年に辞任したときに大統領職に成功しましたが、1976年に完全な任期に選挙で敗北しました。フォードは、大統領または副大統領の選挙を勝ち取らずに米国大統領になった唯一の人物です。
フォードはネブラスカ州オマハで生まれ、ミシガン州グランドラピッズで育ちました。彼はミシガン大学に通い、そこで学校のサッカーチームでプレーしてから最終的にイェールロースクールに通いました。その後、彼は1942年から1946年まで米国海軍保護区に勤務しました。フォードは1949年にミシガン州5の米国代表として政治的キャリアを始めました。
考え3：
```

最後に、LLMは答えを返します。

**応答3：**
```
ジェラルドフォードは1913年に生まれました。1911年は1913年以前です。回答[ロナルドレーガン]
```

##反応チェーンを手動で実行します

このセクションでは、Reactチェーンを段階的に実行します。

次のコードセルには、いくつかのことが必要です。
1. LLMが反応する方法を理解するための指示（コンテキスト）。
2.少なくとも1つの模範。
3. LLMのアクションを実行するツール。
4. LLM呼び出しを行うパームAPIモデルオブジェクト。

In [ ]:
context = """思考、行動、観察で質問に答える。

次に取るべき行動を考える。そして行動を起こす。
すべてのアクションはウィキペディアのルックアップです。
ウィキペディアのアクションはベストマッチの記事の先頭を返します。
ウィキペディアのルックアップをするときは、<STOP>でルックアップを終了します。
ウィキペディアアクションの後、観察があります。
オブザベーションはウィキペディアのルックアップアクションから学んだことに基づいています。
観察が終わったら、再び思考でループを始めます。

必要に応じて、思考、行動、観察を繰り返します。
質問の答えがわかるまで、必要に応じて繰り返す。
答えがわかったと思ったら、その答えを次のような形式で返します：
「Answer[答えはここに角括弧で囲まれています]」という形式で、思考の一部として答えを返します。
答え」は必ず大文字にしてください。

質問に答えるためには、観察結果の情報のみを使用してください。"""

exemplar = """Example:
質問 ロナルド・レーガンとジェラルド・フォード、どちらが先に生まれた？
思考1：ロナルド・レーガンがいつ生まれたのか調べてみよう。
行動1：ロナルド・レーガン<STOP
観察1：ロナルド・ウィルソン・レーガン（Ronald Wilson Reagan、1911年2月6日 - 2004年6月5日）はアメリカの政治家、俳優で、1981年から1989年まで第40代大統領を務めた。保守派で、西海岸出身者初の大統領であり、初の離婚歴のある大統領でもある。イリノイ州タンピコで生まれ、同州ディクソンで育つ。ユーリカ・カレッジで経済学と社会学を学んだ。卒業後はカリフォルニアに移り、ラジオのスポーツアナウンサーとなる。その後、俳優業に転身し、50本以上の映画に出演。レーガンは1947年から1952年まで映画俳優組合の会長を務めた。
思考2：ロナルド・レーガンは1911年生まれ。ジェラルド・フォードがいつ生まれたか調べてみよう。
アクション2：ジェラルド・フォード<STOP
観察2：ジェラルド・ルドルフ・フォード・ジュニア（JERR-əld; 本名レスリー・リンチ・キング・ジュニア、1913年7月14日 - 2006年12月26日）は、1974年から1977年まで第38代アメリカ合衆国大統領を務めたアメリカの政治家。それ以前は1965年から1973年まで下院共和党党首を務め、スピロ・アグニューの辞任後、リチャード・ニクソン大統領によって第40代副大統領に任命された。1974年にニクソンが辞任したため、フォードが大統領職を引き継いだが、1976年の任期満了に伴う選挙では落選した。フォードは、大統領選挙にも副大統領選挙にも当選せずにアメリカ大統領になった唯一の人物である。
ネブラスカ州オマハで生まれ、ミシガン州グランドラピッズで育った。ミシガン大学に入学し、同校のフットボール・チームでプレーした後、エール大学ロースクールに進学。その後、1942年から1946年まで米海軍予備役として勤務。フォードは1949年、ミシガン州下院議員として政治家としてのキャリアをスタートさせた。
思考3：ジェラルド・フォードは1913年生まれ。1911年は1913年より前。答え【ロナルド・レーガン"""

# Code for calling Wikipedia.
import wikipedia
def wiki_tool(query, return_chars = 1000):
  try:
    page = wikipedia.page(query, auto_suggest=False, redirect=True).content
  # If no exact match, take Wikipedia's suggestion.
  except wikipedia.exceptions.PageError as e:
    page = wikipedia.page(query, auto_suggest=True, redirect=True).content
  snippet = page[0:return_chars]
  return snippet

# Initialized PaLM API model.
import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project=PROJECT_ID, location=LOCATION)
# These settings control how deterministic the LLM response is.
parameters = {
    "temperature": 0,
    "max_output_tokens": 256,
    "top_p": 0.8,
    "top_k": 40
}
model = TextGenerationModel.from_pretrained(MODEL_NAME)


最初のLLMコールは、コンテキスト、模範、質問、および最初の思考のラベルです。

各ラインの開始時のアクション/思考/観測ラベルは、チェーンを反応するために重要であり、LLM応答がインターリーブ反応ステップの「スクリプト」に固執する可能性を高めます。

In [ ]:
question = "イプセンの『人形の家』を上演した劇場の開場年は？"
llm_call_1 = f"{context}\n\n{exemplar}\n\nQuestion: {question}\nThought 1:"
print(llm_call_1)

In [ ]:
response_1 = call_llm(model, parameters, llm_call_1)

応答の1行目と2行目は良いです。このモデルは、合理的な思考と適切なアクションを生成しました。

しかし、上記のツールを使用するのと同じように、LLMはごみのテキストを生成し続けます。LLMSは次のトークンを繰り返し予測し、ReactスタイルのLLMコールでは、次のトークンがRLMの残りのReactチェーンの予測であることを忘れないでください。

ツール使用セクションと同じように、追加のテキストが破棄されます。最初の2つの応答線のみが保持されます：「Thought 1」と「アクション1」。

In [ ]:
# Only take the first two lines of the response.
# Splitlines returns a list with an item for each line.
response_1 = response_1.splitlines()[0:2]
# Turn response 1 into text from the list so we can concatenate to llm call 1.
response_1 = ("\n").join(response_1)
print(response_1)

次に、LLMの「アクション1」応答でウィキペディアツールをクエリします。

In [ ]:
# Look up the LLM's action in Wikipedia.
wiki_text_1 = wiki_tool("人形の家")
print(wiki_text_1)

次に、wikipediaツール出力を「観測1」として追加して、次のLLMコールを作成し、「観測2」の考えを追加します。

In [ ]:
# Construct the next LLM call.
llm_call_2 = f"{llm_call_1} {response_1}\nObservation 1: {wiki_text_1}\nThought 2:"
print(llm_call_2)

In [ ]:
response_2 = call_llm(model, parameters, llm_call_2)

Reactチェーンの3回目のLLMコールの場合、2回目の呼び出しと同じ手順に従います。
1. 応答の最初の2行を取ります。
2. ウィキペディアのアクションを調べます。
3. 応答、ウィキペディア出力、および以前のLLMコールからLLMコールを組み立てます。

In [ ]:
# Only take the first two lines of the response.
# Splitlines returns a list with an item for each line.
response_2 = response_2.splitlines()[0:2]
# Turn response 1 into text from the list so we can concatenate to llm call 1.
response_2 = ("\n").join(response_2)
print(response_2)

In [ ]:
# Look up the LLM's action in Wikipedia.
wiki_text_2 = wiki_tool("デンマーク、コペンハーゲンの王立劇場")
print(wiki_text_2)

In [ ]:
# Construct the next LLM call.
llm_call_3 = f"{llm_call_2} {response_2}\nObservation 2: {wiki_text_2}\nThought 3:"
print(llm_call_3)

In [ ]:
response_3 = call_llm(model, parameters, llm_call_3)

そして、私たちには答えがあります！

## Running Reactチェーンのための完全なPythonコードスニペット

アプリケーションでReactを使用するには、以前に手動で実行された手順を自動化する必要があります。

以下の指導的コードスニペットは、Reactチェーンを実行します。LLMへのフォーマットされたReactコールを作成し、アクションを抽出し、アクションを実行し、LLMが回答で応答したかどうかを検出します。

それは**非常に**あなたが以下のコードを歩いて、コメントを読んで、Reactチェーンがどのように自動化されているかをよりよく理解することをお勧めします。

これは生産対応のコードではありません：
1. スニペットは、この特定の最小限のReactの例にハードコードされています。Reactチェーンは異なるように見えます（これについては後で詳しく説明します）、Reactチェーンで構築された有用なアプリケーションにはカスタマイズされたツールが必要です。
2. スニペットは脆い、特にベアボーンウィキペディアツール。
3. LLMは、以前のアクションを再評価し、無限にループを反応させる可能性があります。このスニペットは、「MAX_STEPS」LLMコールの後に停止し、生産Reactコードはループをキャッチして回復しようとするはずです。

In [ ]:
import wikipedia

def call_llm(model, parameters, llm_call, show_activity = True):
  # Wraps an LLM call to Vertex, optionally displaying the call and response.
  response = model.predict(llm_call, **parameters).text

  if show_activity:
    BOLD = "\033[1m"
    UNFORMAT = "\033[0m\x1B[0m"
    print(f"{BOLD}The call to the LLM:{UNFORMAT}\n{llm_call}\n")
    print(f"{BOLD}The response:{UNFORMAT}")
    print(response)
  return response  # Return to `_` if not needed.


def wiki_tool(query, return_chars = 1000):
  try:
    page = wikipedia.page(query, auto_suggest=False, redirect=True).content
  # If no exact match, take Wikipedia's suggestion.
  except wikipedia.exceptions.PageError as e:
    page = wikipedia.page(query, auto_suggest=True, redirect=True).content
  snippet = page[0:return_chars]
  return snippet


def wiki_react_chain(model,
                     parameters,
                     context,
                     exemplar,
                     question,
                     max_steps=7,
                     show_activity=False):
  # Call an LLM in a ReACT-style Thought -> Action -> Observation loop.
  # Call the LLM max_steps times or to an answer in the pattern Answer[ans].

  # Construct the first LLM call, teeing up the first thought.
  next_llm_call = f"{context}\n\n{exemplar}\n\nQuestion: {question}\nThought 1:"

  step = 1
  while step <= max_steps:

    if show_activity:
      print(f"\033[1mReAct chain step {step}:\033[0m\x1B[0m")
    llm_response = call_llm(model, parameters, next_llm_call, show_activity)

    # Check for an answer. Look only at the first line of the response, since
    #   the LLM will continue predicting beyond the next thought.
    # This is brittle, it assumes no line breaks in the thought.
    response_first_line = llm_response.splitlines()[0]
    first_line_answer_split = response_first_line.split("Answer[")
    if len(first_line_answer_split) > 1:  # If there's a split on "Answer[".
      # Return the answer, removing the "]" that comes after the answer.
      return first_line_answer_split[1].split("]")[0]

    # If no answer, assume following response line is action.
    response_second_line = llm_response.splitlines()[1]
    """
      アクションの終わりを示すハードコードされた「<STOP>」文字に注意。
      LLMの最初の行が思考で、2行目が行動だと仮定すれば、これは厳密には必要ない。
      レスポンスの最初の行が思考で、2行目がアクションである。
      それ以降の行はゴミである。しかし、LLMに明示的に次のようなシグナルを送るように指示する。
      LLMに応答構造を明示的に指示することで、より構造的に一貫した
      また、ReActが失敗する可能性のある1つの方法を発見しやすくなる。
    """
    # Extract the wiki query from the action line of the response.
    wiki_query = response_second_line.split(":")[1].split("<STOP>")[0]
    # Remove leading/trailing whitespace.
    wiki_query = wiki_query.strip()
    if show_activity:
      print(f"\033[1mQuerying wikipedia for: {wiki_query}.\033[0m\x1B[0m")
    wiki_text = wiki_tool(wiki_query)

    # Assemble the next LLM call.
    # Only use the lines of the LLM response with the first thought and action.
    usable_response = f"{response_first_line}\n{response_second_line}"
    # Assemble the wiki response into the observation line.
    obs = f"Observation {step}: {wiki_text}"
    step += 1
    # Previous llm call + the first action and thought in the response +
    # the result of the wikipedia lookup = llm call for next ReAct step.
    # Note that next_llm_call was the last call we made, but we reassign it to
    #   the same variable name so the loop works.
    next_llm_call = f"{next_llm_call} {usable_response}\n{obs}\nThought {step}:"

  # If max_steps exceeded and the loop exits.
  # Would be better to raise an exception.
  return None

上記の反応チェーンコードスニペットを使用した例。

In [ ]:
import vertexai
from vertexai.language_models import TextGenerationModel

# Outside this notebook, set PROJECT_ID, LOCATION, and MODEL_NAME.
# When running in the notebook, these are set in part 0.
vertexai.init(project=PROJECT_ID, location=LOCATION)
# These settings control how deterministic the LLM response is.
parameters = {
    "temperature": 0,
    "max_output_tokens": 256,
    "top_p": 0.8,
    "top_k": 40
}
model = TextGenerationModel.from_pretrained(MODEL_NAME)

context = """思考、行動、観察で質問に答える。

次に取るべき行動を考える。そして行動を起こす。
すべてのアクションはwikipediaのルックアップです。
ウィキペディアのアクションはベストマッチする記事の先頭を返します。
ウィキペディアを検索するアクションを起こすときは、<STOP>で検索を終了します。
wikipediaアクションの後、あなたは観察を行います。
観察はwikipedia lookupアクションから学んだことに基づいています。
観察が終わったら、再び思考でループを始めます。

考え、行動し、観察することを必要に応じて繰り返す。
質問の答えがわかるまで、必要に応じて繰り返す。
答えがわかったと思ったら、その答えを次のような形式で返します：
「答え[答えはここに角括弧で囲む]」という形式で答えを返す。
を思考の一部として返します。答え」は必ず大文字で。

観察に含まれる情報は、質問に答えるためにのみ使用してください。"""

exemplar = """Example:
質問 ロナルド・レーガンとジェラルド・フォード、どちらが先に生まれた？
思考1：ロナルド・レーガンがいつ生まれたのか調べてみよう。
行動1：ロナルド・レーガン<STOP
観察1：ロナルド・ウィルソン・レーガン（Ronald Wilson Reagan、1911年2月6日 - 2004年6月5日）はアメリカの政治家、俳優で、1981年から1989年まで第40代大統領を務めた。保守派で、西海岸出身者初の大統領であり、初の離婚歴のある大統領でもある。イリノイ州タンピコで生まれ、同州ディクソンで育つ。ユーリカ・カレッジで経済学と社会学を学んだ。卒業後はカリフォルニアに移り、ラジオのスポーツアナウンサーとなる。その後、俳優業に転身し、50本以上の映画に出演。レーガンは1947年から1952年まで映画俳優組合の会長を務めた。
思考2：ロナルド・レーガンは1911年生まれ。ジェラルド・フォードはいつ生まれたのだろう。
アクション2：ジェラルド・フォード<STOP
観察2：ジェラルド・ルドルフ・フォード・ジュニア（JERR-əld; 本名レスリー・リンチ・キング・ジュニア、1913年7月14日 - 2006年12月26日）は、1974年から1977年まで第38代アメリカ合衆国大統領を務めたアメリカの政治家。それ以前は1965年から1973年まで下院共和党党首を務め、スピロ・アグニューの辞任後、リチャード・ニクソン大統領によって第40代副大統領に任命された。1974年にニクソンが辞任したため、フォードが大統領職を引き継いだが、1976年の任期満了に伴う選挙では落選した。フォードは、大統領選挙にも副大統領選挙にも当選せずにアメリカ大統領になった唯一の人物である。
ネブラスカ州オマハで生まれ、ミシガン州グランドラピッズで育った。ミシガン大学に入学し、同校のフットボール・チームでプレーした後、エール大学ロースクールに進学。その後、1942年から1946年まで米海軍予備役として勤務。フォードは1949年、ミシガン州下院議員として政治家としてのキャリアをスタートさせた。
思考3：ジェラルド・フォードは1913年生まれ。1911年は1913年より前。答え【ロナルド・レーガン"""

question = "フォードT型を設計した3人のエンジニアのうち、最も若いエンジニアが生まれた都市は？"

answer = wiki_react_chain(model,
                          parameters,
                          context,
                          exemplar,
                          question,
                          show_activity = True)
print(answer)


上記の「質問」を変更して実験します。あなたは素晴らしい結果を得ることができないかもしれません。これは、脆いウィキペディアツールによるものかもしれませんが、反応のエラーも表示される場合があります。

コンテキストまたは模範を変更することにより、反応障害のパフォーマンスをどのように改善できるかを考えてください。

##その他の反応のユースケース

Reactパターンは、質問に答えるだけではありません。

異なるコンテキストと模範で、上記のReactコードスニペットは事実チェックに適合しています。

In [ ]:
question = "日本のGDPはBRICSのGDPよりも高い。"

context = """あなたは、主張が真か偽かを検証している。
思考、行動、観察を用いて主張を検証する。
主張を支持する観察があるか、反証する観察があるかを判断する。

その主張を検証するために、次に取るべき行動を考える。そして行動を起こす。
すべてのアクションはウィキペディアのルックアップです。
ウィキペディアのアクションはベストマッチの記事の先頭を返します。
ウィキペディアの検索アクションを行うときは、<STOP>で検索を終了します。
wikipediaアクションの後、あなたは観察を行います。
観察は、ウィキペディアの検索アクションから学んだことに基づく。
観察が終わったら、再び思考でループを始める。

考え、行動し、観察することを必要に応じて繰り返す。
主張について結論が出るまで、必要に応じて繰り返す。
観察によって主張が否定された場合は、答えを「Answer[REFUTES]」として返します。
主張を支持する観察があった場合は、「Answer[SUPPORTS]」として答えを返してください。

観察に含まれる情報は、質問に答えるためにのみ使用してください。"""

exemplar = """Example:
主張 ロナルド・レーガンはジェラルド・フォードより先に生まれた。
思考1：ロナルド・レーガンがいつ生まれたのか調べてみよう。
行動1：ロナルド・レーガン<STOP
観察1：ロナルド・ウィルソン・レーガン（Ronald Wilson Reagan、1911年2月6日 - 2004年6月5日）はアメリカの政治家、俳優で、1981年から1989年まで第40代大統領を務めた。保守派で、西海岸出身者初の大統領であり、初の離婚歴のある大統領でもある。イリノイ州タンピコで生まれ、同州ディクソンで育つ。ユーリカ・カレッジで経済学と社会学を学んだ。卒業後はカリフォルニアに移り、ラジオのスポーツアナウンサーとなる。その後、俳優業に転身し、50本以上の映画に出演。レーガンは1947年から1952年まで映画俳優組合の会長を務めた。
思考2：ロナルド・レーガンは1911年生まれ。ジェラルド・フォードはいつ生まれたのだろう。
アクション 2: ジェラルド フォード<STOP>
観察 2: ジェラルド ルドルフ フォード ジュニア (JERR-əld、本名: レスリー リンチ キング ジュニア、1913 年 7 月 14 日 - 2006 年 12 月 26 日) は、1974 年から 1977 年まで第 38 代アメリカ合衆国大統領を務めたアメリカの政治家です。1965 年から 1973 年まで、共和党の米国下院議員を務め、スピロ アグニューの辞任後、リチャード ニクソン大統領によって第 40 代副大統領に任命されました。フォードは、1974 年にニクソンが辞任したときに大統領職を引き継ぎましたが、1976 年に任期満了の選挙で敗れました。フォードは、大統領または副大統領の選挙に勝たずに米国大統領になった唯一の人物です。
フォードはネブラスカ州オマハで生まれ、ミシガン州グランドラピッズで育った。ミシガン大学に入学し、同校のフットボールチームでプレーした後、最終的にイェール大学ロースクールに進学した。その後、1942年から1946年まで米国海軍予備役に勤務した。フォードは1949年にミシガン州5区の米国下院議員として政治活動を開始した。
考え3: ジェラルド・フォードは1913年に生まれた。ロナルド・レーガンは1911年に生まれた。1911年は1913年より前である。ロナルド・レーガンはジェラルド・フォードより前に生まれた。回答[支持]"""

answer = wiki_react_chain(model,
                          parameters,
                          context,
                          exemplar,
                          question,
                          show_activity = True)
print(answer)


ウィキペディアツールの制限は、このプロンプトのユーティリティを制限し、ニュートラルな「十分な情報」の回答に対するサポートの欠如も同様です。

ただし、このユースケースにどのように簡単に適応したかを考えてください。反応パターンは、次のことでも良い結果を示しています。
* テキストベースの仮想世界とのナビゲートと対話。
* Webをサーフィンします。
* 購入手順を使用して、eコマーストランザクションを作成します。
* ジャーナル記事の文献検索の実施。

<a name="react-tools"> </a>
##ツールの使用ベストプラクティス

上記のプロンプトを実験した場合、おそらく失敗が発生した可能性があります。多くの場合、これはウィキペディアツールが限られているためです。

いくつかのベストプラクティスに従うことで、この教育の例よりも堅牢で効果的なツールを構築するのに役立ちます。

1. Do プロンプト内でツールとその使用方法を明確に記述しましょう。
  * 理想的なツール使用を示すfew-shot事例を含めましょう。
  * 例えば、「doc search」とだけ記述されたツールは、「自然言語クエリを使用して社内文書を検索します。レスポンスは、クエリとの関連性が高い順に並べられた文書名のリストです。」のように記述された同じツールよりもパフォーマンスが低くなります。
2. Do ツールの範囲と複雑さを慎重に検討しましょう。
  * ツールのAPIがLLMにとって十分にシンプルかどうかを検討しましょう。
  * 多くの場合、1つの複雑なツールよりも複数のシンプルなツールの方が効果的です。開発者にとっては単一のAPIであっても、LLMツールとしては複数のツールに分けた方が良い場合があります。
  * 例えば、ユースケースでデータベースにアクセスするためにSQLを実行する必要がある場合、LLMを使用してSQLクエリを最初から生成するのではなく、いくつかの個別のSQLテンプレートを個々のツールとして検討しましょう。
3. Do ツール出力を構造的および文体的に一貫させましょう。
  * ツール出力のバリエーションが少ないほど、LLMがその出力を効果的に使用する可能性が高くなります。
4. Do ツール出力を短く、関連性の高いものにしましょう。
  * 冗長なツール出力は、LLMの入力長制限に負担をかける可能性があります。
  * ReAct論文のWikipediaエージェント実装は素晴らしい例です。Wikipedia記事内を検索し、記事全体ではなく、見つかった用語の周辺のテキストスニペットのみを返します。
5. Do エラーを適切に処理しましょう。
  * 例外をキャッチし、有用なエラーメッセージを提供しましょう。
  * タイムアウトやレート制限などのツール側の問題を管理しましょう。
  * few-shot事例でエラー処理を示しましょう。
  * ツールが失敗し、次のLLM呼び出しで有用なエラーを提供した場合、LLMは自己修正する可能性があります。
6. Do ツール使用プロンプトをチューニングしましょう。
  * 様々なツール使用を含むパラメータ効率の良いチューニングセット（わずか10個の例でも）は、パフォーマンスを大幅に向上させることができます。
7. Do LLMを呼び出してツールアクションを生成する際の出力長を制限しましょう。
  * LLMはツールアクションを超えてテキストを生成し続けます。
8. Don't セキュリティを忘れないでください。多くのツール使用パターンはセキュリティリスクを生み出します。
  * LLMのツールを介してアクセス可能なものはすべて、敵対的な入力で実験するエンドユーザーに見られる可能性があると想定しましょう。
  * LLMのツール呼び出しが悪意のあるものであることはないと想定しないでください。例えば、SQLインジェクションはLLMツールを介して可能です。

このノートブックのツールは、これらのベストプラクティスの多くに従っていません。

1. Wikipediaの記事は構造が予測できません。
1. Wikipediaの記事は数千語になることもありますが、このツールは記事の関連部分に焦点を当てることをサポートしていません。
1. プロンプトではWikipediaとは何か、またはその使用方法が説明されていません（ただし、LLMはトレーニングデータからWikipediaが何かを「知って」います）。
1. エラーメッセージがなく、エラー処理は最小限です。


## 利点を反応します

1. 幻覚が少ない。
  * 信頼できる情報源とLLMの「メモリ」に依存する接地。
1. 再訓練なしでLLMの知識を更新/拡張します。
1. 既製のLLMSで動作し、追加のLLMトレーニングやチューニングは必要ありません。
1. さまざまなユースケースをサポートします。
1. 複数のツールで動作します。
1. ツールを改善することでシステム全体のパフォーマンスを改善することは、プロンプトまたはLLM自体を改善するよりも簡単です。

## React Disdvantages

1. 複数のLLM呼び出しにより、遅い（高レイテンシー）および高価です。
1. 外部ツールは、維持およびセキュリティの懸念を維持し、より多くのシステムコンポーネントを意味します。
1. 反応ループとその他の非回答シナリオが一般的です。
  * Vs.幻覚がより一般的であると思考の連鎖。
  * 専門的または最新の情報を必要としないユースケースの場合、思考の連鎖は反応する可能性があります。
1. 反応推論（Think-> Act）は柔軟性が低く、純粋な一連の思考の連鎖のより柔軟な推論とパフォーマンスが低下する可能性があります。
1. 外部情報が必要な場合、RAGよりも複雑な場合、検索がLLMによって制御されない場合にアプローチします。
1. ツール統合を超えて、追加の機能が必要です。
  * ループ救済。
  * ツールエラーの管理。
  * 思考のチェーンフォールバック。

##ベストプラクティスを反応します

[ツール使用](https://github.com/GoogleCloudPlatform/specialized-training-content/blob/184be57c9ff4de18e20f3fdfbe1ef7fb35ce023f/courses/generative_ai/app_dev_llm/#react-tools)上記のベストプラクティスを超えて。

1. Don't temperature=0を盲目的に使用しないでください。
  * タスクやツールの記述方法によってパフォーマンスが大きく変わる可能性があります。
  * "Thought"、"Action"、"Observation"以外のラベルとステップのスキップを含む事例を試してみましょう。
  * 様々な思考/推論と行動スタイルの事例を試してみましょう。例えば：
    * 次のアクションを特定する思考が最適なタスクもあれば、最初の思考で完全な計画を立てるのが最適なタスクもあります。
    * 無関係な観察やツールエラーの後に計画を調整したり、前の思考を再考したりする思考/行動を示しましょう。
  * 直前の観察の最も重要な部分を言い換える思考を試してみましょう。
1. Do ReActチェーンがループに陥るのをキャッチしましょう。
  * ループのキャッチを示す事例で実験しましょう。
  * 繰り返される行動をキャッチし、繰り返される行動を指摘する観察をLLMに返すことを検討しましょう。LLMは回復できるかもしれません。
  * temperature > 0でループしているチェーンを再実行してみましょう。
  * ReActが研究ベンチマークデータセットで最先端である場合、それは多くの場合、連鎖思考の自己整合性フォールバックを伴います。
1. Do ファインチューニングを活用しましょう。
  * ReActチェーン全体にわたるチューニング事例を含め、最初または最後のLLM呼び出しの事例だけでなく、全体を含めましょう。
  * エラー/失敗処理をチューニングデータに含めましょう。
  * 最終的な答えが正しい場合でも、不正確なReAct推論を含むチューニング事例を使用しないでください。
1. Don't より単純な代替案を評価せずにReActを実装しないでください。
  * 管理された拡張機能/プラグインを検討しましょう。
    * 拡張機能サービスは、セキュリティ、可観測性、監視、評価などを提供し、実装作業を削減する可能性があります。
    * 技術的な評価なしに、管理された拡張機能/プラグインサービスがニーズを満たすと想定しないでください。
  * 外部知識をLLM呼び出しに統合するより簡単な方法を検討しましょう。（例：上記のRAGパターン1）。
1. Do 大規模なReActのデバッグにLLMを使用しましょう。
  * LLMに、タイプ別に失敗を分類する（例：推論ミス、ツール検索失敗、ループに陥った）および/またはReActチェーンの各ステップを正しいか間違っているかを識別するように促します。
1. Do テスト、パフォーマンス測定（ドリフトを含む）、システム監視、CI/CDなどにツール機能を含めましょう。

# パート4：LangchainとReact
<img src = "https://raw.githubusercontent.com/GoogleCloudPlatform/specialized-training-content/184be57c9ff4de18e20f3fdfbe1ef7fb35ce023f/courses/generative_ai/app_dev_llm/images/5-chained.png">

Langchain は、LLM をすぐに始めるための優れたライブラリです。多くの[ツールの統合](https://python.langchain.com/docs/integrations/tools/)や組み込みの [ReAct エージェント](https://python.langchain.com/docs/modules/agents/agent_types/react)など、さまざまな便利な機能が備わっています。

ただし、Langchainとの反応は、すべてのユースケースに最適ではない場合があります。Langchainを使用して使用する場合は、ニーズを満たしているかどうかを評価することが重要です。

Langchainが現在ユースケースのニーズを満たしていないことがわかった場合でも、Langchainが1.0リリースに近づくと機能が追加されることに注意してください。

Langchainには、[Langsmith](https://docs.smith.langchain.com/)という名前で利用可能な独自の評価と生産ツールもあります。

##基本的なLangchain React Agent

LangchainでのReactの主な利点は、開始するのが非常に少ない作業であることです。

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import VertexAI
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
import wikipedia
import vertexai

# This is the langchain connection to Vertex AI.
# Note this depends on vertexai.init (which was run in Part 0).
llm = VertexAI(model_name=MODEL_NAME, temperature=0)

# Initialize the Wikipedia tool.
_ = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
# This next line invisibly maps to the previous line. The WikipediaQueryRun
#   call is what matters here for Langchain to use its "wikipedia", not
#   the variable that call is output to.
tools = load_tools(["wikipedia"], llm=llm)

# Create the ReAct agent.
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

# You can change this question to see how the agent performs.
# You may get a GuessedAtParserWarning from the wikipedia API, ignore it.
agent.run("ボンゾのベッドタイム』でチンパンジーと共演したアメリカ大統領とは？")

Langchain のもう 1 つの優れた機能は、組み込みの[ツール統合](https://python.langchain.com/docs/integrations/tools/)です。特に便利なツールの 1 つは数学用です。LLM は数学が苦手ですが、外部計算機があれば数学のパフォーマンスが向上します。

In [ ]:
# The answer is 4489.
# This may timeout or error, that's ok.
agent.run("What's 67^2?")

In [ ]:
# Make the llm-math tool available to the agent.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)
agent.run("What's 67^2?")

##観測可能性の課題

デフォルトでは、LangchainはReactチェーンの最終出力のみを返します。しかし、特にデバッグするときは、すべてのLLMコールを見ることが必要な場合があります。

Langchainには、基礎となるLLMコールにある程度の観測性を提供する冗長モードが含まれています。

In [ ]:
# Note verbose is part of the agent declaration, not the run.
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)

agent.run("ボンゾのベッドタイム』でチンパンジーと共演したアメリカ大統領とは？")

ここで、冗長モードは、最初の考えでは、LLMがその内部知識を使用したことを示しています。

しかし、エージェントがどのように回答に到達したか、またはエージェントが失敗した理由を理解するのに冗長モードは常に十分ではありません。

In [ ]:
agent.run("2010年9月1日は何曜日でしたか？")

完全にデバッグするには、Langchain 内部の可視性を向上させる必要があります。 このカスタム可観測性コードのスニペット (この[ノートブック](https://github.com/GoogleCloudPlatform/specialized-training-content/blob/184be57c9ff4de18e20f3fdfbe1ef7fb35ce023f/courses/generative_ai/langchain_observability_snippet/langchain-observability-snippet.ipynb)からのもの) は、Langchain の[コールバック ハンドラー](https://python.langchain.com/docs/modules/callbacks/)を使用して、エージェントの実行時に何が起こるかを正確に示します。

In [ ]:
# @title
# Import dependencies.
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import AgentAction, AgentFinish, Document, LLMResult
import pdb
from prettyprinter import cpprint
from typing import Any, Dict, List, Optional, Sequence, Type, Union
from uuid import UUID

# Two helper classes.
class Color():
  """印刷された色をより簡単に理解し、より速く操作する。"""
  PURPLE = "\033[95m"
  CYAN = "\033[96m"
  DARKCYAN = "\033[36m"
  BLUE = "\033[94m"
  GREEN = "\033[92m"
  YELLOW = "\033[93m"
  RED = "\033[91m"
  BOLD = "\033[1m"
  UNDERLINE = "\033[4m"
  ITALICS = "\x1B[3m"
  END = "\033[0m\x1B[0m"


class OutputFormatter:
  """ コールバックから出力される出力の形式を制御するヘルパー クラス。

prod で使用する場合は、出力が書き込まれる場所のハードコーディングを削除する方法で再実装することを検討してください。
Python ログを使用してから、カスタム構成を渡すこともできます。
  """
  # TODO: Add str casting here to reduce f"{}" in callback class to this class.
  def heading(text: str) -> None:
    print(f"{Color.BOLD}{text}{Color.END}")

  def key_info(text: str) -> None:
    print(f"{Color.BOLD}{Color.DARKCYAN}{text}{Color.END}")

  def key_info_labeled(label: str,
                       contents: str,
                       contents_newlined: Optional[bool] = False
                       ) -> None:
    print(f"{Color.BOLD}{Color.DARKCYAN}{label}: {Color.END}{Color.DARKCYAN}",
          end="")
    if contents_newlined:
      contents = contents.splitlines()
    cpprint(f"{contents}")
    print(f"{Color.END}", end="")

  def debug_info(text: str) -> None:
    print(f"{Color.BLUE}{text}{Color.END}")

  def debug_info_labeled(label: str,
                         contents: str,
                         contents_newlined: Optional[bool] = False
                         ) -> None:
    print(f"{Color.BOLD}{Color.BLUE}{label}: {Color.END}{Color.BLUE}",
          end="")
    if contents_newlined:
      contents = contents.splitlines()
    cpprint(f"{contents}")
    print(f"{Color.END}", end="")

  def llm_call(text: str) -> None:
    print(f"{Color.ITALICS}{text}{Color.END}")

  def llm_output(text: str) -> None:
    print(f"{Color.UNDERLINE}{text}{Color.END}")

  def tool_call(text: str) -> None:
    print(f"{Color.ITALICS}{Color.PURPLE}{text}{Color.END}")

  def tool_output(text: str) -> None:
    print(f"{Color.UNDERLINE}{Color.PURPLE}{text}{Color.END}")

  def debug_error(text: str) -> None:
    print(f"{Color.BOLD}{Color.RED}{text}{Color.END}")

# Actual langchain callback handler, this produces status updates during a
#   langchain execution.
class AllChainDetails(BaseCallbackHandler):
  """チェーンの進行状況と状態の詳細を出力する。

  チェーンの各実行ステップのコールバック時に利用可能な詳細を公開する。

  このクラスのメソッドの引数は、コールバック・メソッドで利用可能な引数（の大部分？
    に基づいています。
    クラスのすべての実装がすべての引数を使用するわけではありません。

  使い方
    使用法：コールバック・ハンドラを受け付けるlangchainメソッドまたはクラスの引数として渡す。
      ハンドラを受け付けるメソッドやクラスに引数として渡します。全てのlangchainクラスが、初期化時にコールバックハンドラが提供された場合、全てのコールバックを呼び出すわけではないことに注意してください。
      初期化時にコールバックハンドラが提供された場合、全てのlangchainクラスが全てのコールバックを起動するわけではないことに注意してください。
      推奨される使い方は、チェーンを実行するときにコールバックハンドラを提供することです。
      チェーンを実行するときにコールバックハンドラを提供することです。

  Example:
    from langchain import LLMChain, PromptTemplate
    from langchain.llms import VertexAI
    import vertexai  # Comes from google-cloud-aiplatform package.
    vertexai.init(project=PROJECT_ID, location=REGION)

    llm = VertexAI(temperature=0)  # Use any LLM.
    prompt_template = "What food pairs well with {food}?"
    handler = AllChainDetails()
    llm_chain = LLMChain(
      llm=llm,
      prompt=PromptTemplate.from_template(prompt_template))
    llm_chain("chocolate", callbacks=[handler])

  Args:
    debug_mode: True の場合、各チェーン ステップの詳細を出力し、予期しない動作が検出されたときにブレークポイント (pdb を使用) をアクティブにします。
    ブレークポイントはコールバック内にあるため、検査可能な langchain 状態の量は、langchain がコールバックに表示するものに制限されることに注意してください。
  out: 出力を管理するためのクラス。このクラスに付随する OutputFormatter でのみテストされます。
  """
  def __init__(self,
               debug_mode: Optional[bool] = False,
               out: Type[OutputFormatter] = OutputFormatter,
               ) -> None:
    self.debug_mode = debug_mode
    self.out = out

  def on_llm_start(self,
                   serialized: Dict[str, Any],
                   prompts: List[str],
                   **kwargs: Any) -> None:
    """Run when langchain calls an LLM."""
    self.out.heading(f"\n\n> Sending text to the LLM.")

    if len(prompts) > 1:
      self.out.debug_error("prompts has multiple items.")
      self.out.debug_error("Only outputting first item in prompts.")
      if self.debug_mode:
        self.out.debug_info_labeled("Prompts", f"{prompts}")
        pdb.set_trace()

    self.out.key_info(f"Text sent to LLM:")
    self.out.llm_call(prompts[0])

    if self.debug_mode:
      self.out.debug_info_labeled("Arguments", f"{kwargs}")
      self.out.debug_info_labeled("serialized", f"{serialized}")

  def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
    """Run after LLM response is received by langchain."""
    self.out.heading(f"\n\n> Received response from LLM.")

    if len(response.generations) > 1:
      self.out.debug_error("response object has multiple generations.")
      self.out.debug_error("Only outputting first generation in response.")
      if self.debug_mode:
        self.out.debug_info_labeled("response", f"{response}")
        pdb.set_trace()

    self.out.key_info(f"Text received from LLM:")
    self.out.llm_output(response.generations[0][0].text)

    if self.debug_mode:
      self.out.debug_info_labeled("Arguments", f"{kwargs}")
      self.out.debug_info_labeled("response", f"{response}")

  def on_tool_start(self,
                    serialized: Dict[str, Any],
                    input_str: str,
                    **kwargs: Any,) -> None:
    """Run when making a call to a tool."""
    self.out.heading(f"\n\n> Using tool.")
    self.out.key_info_labeled(f"Tool name", f"{serialized['name']}")
    self.out.key_info(f"Query sent to tool:")
    self.out.tool_call(input_str)

    if self.debug_mode:
      self.out.debug_info_labeled("Arguments", f"{kwargs}")
      self.out.debug_info_labeled("serialized", f"{serialized}")

  def on_tool_end(
      self,
      output: str,
      color: Optional[str] = None,
      observation_prefix: Optional[str] = None,
      llm_prefix: Optional[str] = None,
      **kwargs: Any,) -> None:
    """Run on response from a tool."""
    self.out.heading(f"\n\n> Received tool output.")
    self.out.key_info_labeled(f"Tool name", f"{kwargs['name']}")

    if "output" not in locals():
      self.out.debug_error("No tool output.")
      if self.debug_mode:
        pdb.set_trace()
    else:
      self.out.key_info("Response from tool:")
      self.out.tool_output(f"{output}")

    if self.debug_mode:
      self.out.debug_info_labeled("Arguments", f"{kwargs}")
      self.out.debug_info_labeled("observation_prefix",
                                  f"{observation_prefix}")
      self.out.debug_info_labeled("llm_prefix",
                                  f"{llm_prefix}")

  def on_agent_action(self,
                      action: AgentAction,
                      color: Optional[str] = None,
                      **kwargs: Any) -> Any:
    """Run when agent performs an action."""
    self.out.heading(f"\n\n> Agent taking an action.")

    if self.debug_mode:
      self.out.debug_info_labeled("Arguments", f"{kwargs}")
      self.out.debug_info_labeled("action", f"{action}")

  def on_agent_finish(self,
                      finish: AgentFinish,
                      color: Optional[str] = None,
                      **kwargs: Any) -> None:
    """Run after agent completes."""
    self.out.heading(f"\n\n> Agent has finished.")

    if self.debug_mode:
      self.out.debug_info_labeled("Arguments", f"{kwargs}")
      self.out.debug_info_labeled("finish",
                                  f"{finish}")

  def on_llm_error(self,
                   error: Union[Exception, KeyboardInterrupt],
                   **kwargs: Any) -> None:
    self.out.debug_error("LLM Error")
    self.out.debug_info_labeled("Error object", f"{error}")
    if self.debug_mode:
      pdb.set_trace()

  def on_chain_error(self,
                     error: Union[Exception, KeyboardInterrupt],
                     **kwargs: Any) -> None:
    self.out.debug_error("Chain Error")
    self.out.debug_info_labeled("Error object", f"{error}")
    if self.debug_mode:
      pdb.set_trace()

  def on_tool_error(self,
                    error: Union[Exception, KeyboardInterrupt],
                    **kwargs: Any) -> None:
    self.out.debug_error("Chain Error")
    self.out.debug_info_labeled("Error object", f"{error}")
    if self.debug_mode:
      pdb.set_trace()

カスタム観測可能性コードを含むエージェントを使用して、失敗したクエリを繰り返します。

In [ ]:
handler = AllChainDetails()
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)
agent.run("2010年9月1日は何曜日でしたか？",
          callbacks=[handler])

LLMに送信された正確な呼び出しが表示され、LLMがツール（「ツールを使用する」）を選択したとき、LLMのツールへの入力（「ツールに送信されたクエリ：」）、および次のLLMアクティビティが表示されます。

エラーの性質は明確になりました。数学ツールは、LLMに「numexpr」ライブラリで実行する式を作成するように指示しますが、LLMには誤って式に「DateTime」ライブラリが含まれています。

さらに、LLMは、Langchainを呼び出して、ツールの説明や正確なReact実装（標準の思考 - >アクション - >観測とは異なる）を含む反応を実行するために使用します。

### Langchainでの生産観測可能性

安定した生産LLMシステムを実行するには、おそらく集中型の外部ロギング/監視プラットフォームで、強力な観察可能性とロギングが必要です。これがなければ、システムが正しく実行されていることを確認することはできず、デバッグできない場合があります。

Langchainのコールバックの実装はここで役立ち、一部のMLプラットフォームベンダーはLangchainコールバックハンドラーを提供しています。

ただし、一部のユースケースでは、カスタムラングチェーンコールバックハンドラーを作成する必要があります。また、システムに依存しているラングチェーンモジュールの他の部分に応じて、必要な情報をコールバックに表現するためにLangchain内部を変更する必要があります。

##ツールのカスタマイズ摩擦

Langchainエージェントに「DateTime」サポートを追加する方法は次のとおりです。

1. 数学ツールがReactプロンプトで説明されている方法を変更するため、LLMは「DateTime」を使用しないことを知っています。
1. DateTime Operations専用の新しいツールを作成し、LLMが利用できるようにします。
1. Langchain Mathツールを変更して、「DateTime」サポートを追加します。
1. langchain数学ツールを変更して、「numexpr」から例外をキャッチし、次のコールでLLMにエラーメッセージを提供して、LLMが別のアクションを実行できるようにします。

これらには、Langchain内部の知識や、まだ文書化されていないLangchain機能を使用する必要があります。

さらに、最良の反応性能のために、指示、模範、およびツールの説明を調整する必要があります。これは、「DateTime」ツールの問題を管理する以外に、[カスタムLangchainエージェント](https://python.langchain.com/docs/modules/agents/)を作成する必要があることを意味します。

多くのユースケースでは、この摩擦は克服する価値があります。しかし、フレームワークを採用するという決定と同様に、ソフトウェア開発のベストプラクティスに従い、利用可能なフレームワークと構築の長所と短所をゼロから完全に調査します。